In [53]:
import numpy as np
import random, math
import copy, sys

A = np.array([[math.sqrt(2),  -math.sqrt(3)], [math.sqrt(5), math.sqrt(7)]])
x_fin= np.array([math.pi*pow(2,10), -pow(2,5)*math.exp(1)])

b = np.dot(A, x_fin)
qubits = 1

print("Matrix A:")
print(A)
print("\nSolution vector x:")
print(x_fin)
print("\nVector b:")
print(b)

Matrix A:
[[ 1.41421356 -1.73205081]
 [ 2.23606798  2.64575131]]

Solution vector x:
[3216.99087728  -86.98501851]

Vector b:
[4700.17460023 6963.26955782]


In [54]:
Center = np.array([0,0])
Curr_Pow = 20
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  20
Solution x is 3.21699087727594815078E+03
Current  x is 0.00000000000000000000E+00
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 3.21699087727594815078E+03
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ 7.64998814e+12  0.00000000e+00  7.62311212e+12 -7.62311212e+12]
 [ 0.00000000e+00  7.74317465e+12 -7.62311212e+12  7.62311212e+12]
 [ 0.00000000e+00  0.00000000e+00  1.09735531e+13  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.10166795e+13]]

Minimum energy is  -70578764.2075019



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):7649988140613.805, ('q2','q2'):7743174648250.198, ('q3','q3'):10973553070755.344, ('q4','q4'):11016679484764.656}

quadratic = {('q1','q3'):7623112116120.794, ('q1','q4'):-7623112116120.794, ('q2','q3'):-762311

In [55]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):7649988140613.805, ('q2','q2'):7743174648250.198, ('q3','q3'):10973553070755.344, ('q4','q4'):11016679484764.656}

quadratic = {('q1','q3'):7623112116120.794, ('q1','q4'):-7623112116120.794, ('q2','q3'):-7623112116120.794, ('q2','q4'):7623112116120.794} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [56]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  19
Solution x is 3.21699087727594815078E+03
Current  x is 0.00000000000000000000E+00
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 3.21699087727594815078E+03
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ 1.90084872e+12  0.00000000e+00  1.90577803e+12 -1.90577803e+12]
 [ 0.00000000e+00  1.94744198e+12 -1.90577803e+12  1.90577803e+12]
 [ 0.00000000e+00  0.00000000e+00  2.73799747e+12  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.75956067e+12]]

Minimum energy is  -70578764.2075019



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1900848721698.902, ('q2','q2'):1947441975517.0986, ('q3','q3'):2737997465937.6714, ('q4','q4'):2759560672942.329}

quadratic = {('q1','q3'):1905778029030.1985, ('q1','q4'):-1905778029030.1985, ('q2','q3'):-1905

In [57]:
linear = {('q1','q1'):1900848721698.902, ('q2','q2'):1947441975517.0986, ('q3','q3'):2737997465937.6714, ('q4','q4'):2759560672942.329}

quadratic = {('q1','q3'):1905778029030.1985, ('q1','q4'):-1905778029030.1985, ('q2','q3'):-1905778029030.1985, ('q2','q4'):1905778029030.1985} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4            energy num_oc. chain_.
0  0  0  0  0               0.0     999     0.0
1  1  0  0  0 1900848721698.902       1     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [58]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  18
Solution x is 3.21699087727594815078E+03
Current  x is 0.00000000000000000000E+00
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 3.21699087727594815078E+03
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ 4.69388024e+11  0.00000000e+00  4.76444507e+11 -4.76444507e+11]
 [ 0.00000000e+00  4.92684651e+11 -4.76444507e+11  4.76444507e+11]
 [ 0.00000000e+00  0.00000000e+00  6.81803966e+11  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.92585569e+11]]

Minimum energy is  -70578764.2075019



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):469388023697.4509, ('q2','q2'):492684650606.54926, ('q3','q3'):681803965608.8356, ('q4','q4'):692585569111.1646}

quadratic = {('q1','q3'):476444507257.5496, ('q1','q4'):-476444507257.5496, ('q2','q3'):-4764445

In [59]:
linear = {('q1','q1'):469388023697.4509, ('q2','q2'):492684650606.54926, ('q3','q3'):681803965608.8356, ('q4','q4'):692585569111.1646}

quadratic = {('q1','q3'):476444507257.5496, ('q1','q4'):-476444507257.5496, ('q2','q3'):-476444507257.5496, ('q2','q4'):476444507257.5496} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4             energy num_oc. chain_.
0  0  0  0  0                0.0     998     0.0
1  0  1  0  0 492684650606.54926       2     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [61]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  17
Solution x is 3.21699087727594815078E+03
Current  x is 0.00000000000000000000E+00
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 3.21699087727594815078E+03
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ 1.14434928e+11  0.00000000e+00  1.19111127e+11 -1.19111127e+11]
 [ 0.00000000e+00  1.26083241e+11 -1.19111127e+11  1.19111127e+11]
 [ 0.00000000e+00  0.00000000e+00  1.69103291e+11  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.74494093e+11]]

Minimum energy is  -70578764.2075019



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):114434927560.72545, ('q2','q2'):126083241015.2746, ('q3','q3'):169103290964.4178, ('q4','q4'):174494092715.58224}

quadratic = {('q1','q3'):119111126814.3874, ('q1','q4'):-119111126814.3874, ('q2','q3'):-119111

In [62]:
linear = {('q1','q1'):114434927560.72545, ('q2','q2'):126083241015.2746, ('q3','q3'):169103290964.4178, ('q4','q4'):174494092715.58224}

quadratic = {('q1','q3'):119111126814.3874, ('q1','q4'):-119111126814.3874, ('q2','q3'):-119111126814.3874, ('q2','q4'):119111126814.3874} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4            energy num_oc. chain_.
0  0  0  0  0               0.0     999     0.0
1  0  1  0  0 126083241015.2746       1     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [63]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  16
Solution x is 3.21699087727594815078E+03
Current  x is 0.00000000000000000000E+00
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 3.21699087727594815078E+03
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ 2.71526927e+10  0.00000000e+00  2.97777817e+10 -2.97777817e+10]
 [ 0.00000000e+00  3.29768494e+10 -2.97777817e+10  2.97777817e+10]
 [ 0.00000000e+00  0.00000000e+00  4.16019725e+10  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.42973734e+10]]

Minimum energy is  -70578764.2075019



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):27152692708.36272, ('q2','q2'):32976849435.637287, ('q3','q3'):41601972522.208885, ('q4','q4'):44297373397.791115}

quadratic = {('q1','q3'):29777781703.59685, ('q1','q4'):-29777781703.59685, ('q2','q3'):-29777

In [64]:
linear = {('q1','q1'):27152692708.36272, ('q2','q2'):32976849435.637287, ('q3','q3'):41601972522.208885, ('q4','q4'):44297373397.791115}

quadratic = {('q1','q3'):29777781703.59685, ('q1','q4'):-29777781703.59685, ('q2','q3'):-29777781703.59685, ('q2','q4'):29777781703.59685} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [65]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  15
Solution x is 3.21699087727594815078E+03
Current  x is 0.00000000000000000000E+00
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 3.21699087727594815078E+03
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ 6.06015359e+09  0.00000000e+00  7.44444543e+09 -7.44444543e+09]
 [ 0.00000000e+00  8.97223195e+09 -7.44444543e+09  7.44444543e+09]
 [ 0.00000000e+00  0.00000000e+00  1.00635680e+10  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.14112685e+10]]

Minimum energy is  -70578764.2075019



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):6060153586.18136, ('q2','q2'):8972231949.818644, ('q3','q3'):10063568021.104445, ('q4','q4'):11411268458.895557}

quadratic = {('q1','q3'):7444445425.899213, ('q1','q4'):-7444445425.899213, ('q2','q3'):-7444445

In [66]:
linear = {('q1','q1'):6060153586.18136, ('q2','q2'):8972231949.818644, ('q3','q3'):10063568021.104445, ('q4','q4'):11411268458.895557}

quadratic = {('q1','q3'):7444445425.899213, ('q1','q4'):-7444445425.899213, ('q2','q3'):-7444445425.899213, ('q2','q4'):7444445425.899213} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [67]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  14
Solution x is 3.21699087727594815078E+03
Current  x is 0.00000000000000000000E+00
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 3.21699087727594815078E+03
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ 1.15102860e+09  0.00000000e+00  1.86111136e+09 -1.86111136e+09]
 [ 0.00000000e+00  2.60706778e+09 -1.86111136e+09  1.86111136e+09]
 [ 0.00000000e+00  0.00000000e+00  2.34742945e+09  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.02127967e+09]]

Minimum energy is  -70578764.2075019



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1151028601.0906792, ('q2','q2'):2607067782.9093213, ('q3','q3'):2347429450.5522223, ('q4','q4'):3021279669.4477787}

quadratic = {('q1','q3'):1861111356.4748032, ('q1','q4'):-1861111356.4748032, ('q2','q3'):-18

In [68]:
linear = {('q1','q1'):1151028601.0906792, ('q2','q2'):2607067782.9093213, ('q3','q3'):2347429450.5522223, ('q4','q4'):3021279669.4477787}

quadratic = {('q1','q3'):1861111356.4748032, ('q1','q4'):-1861111356.4748032, ('q2','q3'):-1861111356.4748032, ('q2','q4'):1861111356.4748032} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [69]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  13
Solution x is 3.21699087727594815078E+03
Current  x is 0.00000000000000000000E+00
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 3.21699087727594815078E+03
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ 1.05752253e+08  0.00000000e+00  4.65277839e+08 -4.65277839e+08]
 [ 0.00000000e+00  8.33771843e+08 -4.65277839e+08  4.65277839e+08]
 [ 0.00000000e+00  0.00000000e+00  5.02626085e+08  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  8.39551195e+08]]

Minimum energy is  -70578764.2075019



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):105752252.54533958, ('q2','q2'):833771843.4546607, ('q3','q3'):502626085.2761109, ('q4','q4'):839551194.7238892}

quadratic = {('q1','q3'):465277839.1187008, ('q1','q4'):-465277839.1187008, ('q2','q3'):-4652778

In [70]:
linear = {('q1','q1'):105752252.54533958, ('q2','q2'):833771843.4546607, ('q3','q3'):502626085.2761109, ('q4','q4'):839551194.7238892}

quadratic = {('q1','q3'):465277839.1187008, ('q1','q4'):-465277839.1187008, ('q2','q3'):-465277839.1187008, ('q2','q4'):465277839.1187008} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4          energy num_oc. chain_.
0  0  0  0  0             0.0     977     0.0
1  1  0  0  0 105752252.54534      23     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [71]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  12
Solution x is 3.21699087727594815078E+03
Current  x is 0.00000000000000000000E+00
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 3.21699087727594815078E+03
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[-6.45643857e+07  0.00000000e+00  1.16319460e+08 -1.16319460e+08]
 [ 0.00000000e+00  2.99445410e+08 -1.16319460e+08  1.16319460e+08]
 [ 0.00000000e+00  0.00000000e+00  8.35408826e+07  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.52003437e+08]]

Minimum energy is  -70578764.2075019



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-64564385.72733023, ('q2','q2'):299445409.7273303, ('q3','q3'):83540882.63805544, ('q4','q4'):252003437.3619446}

quadratic = {('q1','q3'):116319459.7796752, ('q1','q4'):-116319459.7796752, ('q2','q3'):-1163194

In [72]:
linear = {('q1','q1'):-64564385.72733023, ('q2','q2'):299445409.7273303, ('q3','q3'):83540882.63805544, ('q4','q4'):252003437.3619446}

quadratic = {('q1','q3'):116319459.7796752, ('q1','q4'):-116319459.7796752, ('q2','q3'):-116319459.7796752, ('q2','q4'):116319459.7796752} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4          energy num_oc. chain_.
0  1  0  0  0 -64564385.72733     999     0.0
1  0  0  0  0             0.0       1     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [73]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  12
Solution x is 3.21699087727594815078E+03
Current  x is 4.09600000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is -8.79009122724051849218E+02
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ 1.70316638e+08  0.00000000e+00  1.16319460e+08 -1.16319460e+08]
 [ 0.00000000e+00  6.45643857e+07 -1.16319460e+08  1.16319460e+08]
 [ 0.00000000e+00  0.00000000e+00  1.99860342e+08  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.35683978e+08]]

Minimum energy is  -6014378.480171669



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):170316638.2726698, ('q2','q2'):64564385.72733025, ('q3','q3'):199860342.41773063, ('q4','q4'):135683977.58226937}

quadratic = {('q1','q3'):116319459.7796752, ('q1','q4'):-116319459.7796752, ('q2','q3'):-1163

In [74]:
linear = {('q1','q1'):170316638.2726698, ('q2','q2'):64564385.72733025, ('q3','q3'):199860342.41773063, ('q4','q4'):135683977.58226937}

quadratic = {('q1','q3'):116319459.7796752, ('q1','q4'):-116319459.7796752, ('q2','q3'):-116319459.7796752, ('q2','q4'):116319459.7796752} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4         energy num_oc. chain_.
0  0  0  0  0            0.0     999     0.0
1  0  1  0  0 64564385.72733       1     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [75]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  11
Solution x is 3.21699087727594815078E+03
Current  x is 4.09600000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is -8.79009122724051849218E+02
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ 55798191.13633489         0.          29079864.9449188
  -29079864.9449188 ]
 [        0.           2922064.86366512 -29079864.9449188
   29079864.9449188 ]
 [        0.                 0.          57987131.20886531
          0.        ]
 [        0.                 0.                 0.
   25898948.79113469]]

Minimum energy is  -6014378.480171669



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):55798191.13633489, ('q2','q2'):2922064.863665119, ('q3','q3'):57987131.20886531, ('q4','q4'):25898948.791134693}

quadratic = {('q1','q3'):29079864.9449188, ('q1'

In [76]:
linear = {('q1','q1'):55798191.13633489, ('q2','q2'):2922064.863665119, ('q3','q3'):57987131.20886531, ('q4','q4'):25898948.791134693}

quadratic = {('q1','q3'):29079864.9449188, ('q1','q4'):-29079864.9449188, ('q2','q3'):-29079864.9449188, ('q2','q4'):29079864.9449188} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4         energy num_oc. chain_.
0  0  0  0  0            0.0     922     0.0
1  0  1  0  0 2922064.863665      78     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [77]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  10
Solution x is 3.21699087727594815078E+03
Current  x is 4.09600000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is -8.79009122724051849218E+02
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[20559063.56816744        0.          7269966.2362297  -7269966.2362297 ]
 [       0.         -5878999.56816744 -7269966.2362297   7269966.2362297 ]
 [       0.                0.         18507805.60443265        0.        ]
 [       0.                0.                0.          2463714.39556735]]

Minimum energy is  -6014378.480171669



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):20559063.568167444, ('q2','q2'):-5878999.5681674415, ('q3','q3'):18507805.604432654, ('q4','q4'):2463714.3955673464}

quadratic = {('q1','q3'):7269966.2362297, ('q1','q4'):-726

In [78]:
linear = {('q1','q1'):20559063.568167444, ('q2','q2'):-5878999.5681674415, ('q3','q3'):18507805.604432654, ('q4','q4'):2463714.3955673464}

quadratic = {('q1','q3'):7269966.2362297, ('q1','q4'):-7269966.2362297, ('q2','q3'):-7269966.2362297, ('q2','q4'):7269966.2362297} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4          energy num_oc. chain_.
0  0  1  0  0 -5878999.568167    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [79]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  10
Solution x is 3.21699087727594815078E+03
Current  x is 3.07200000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 1.44990877275948150782E+02
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ 5878999.56816744        0.          7269966.2362297  -7269966.2362297 ]
 [       0.          8801064.43183256 -7269966.2362297   7269966.2362297 ]
 [       0.                0.         11237839.36820295        0.        ]
 [       0.                0.                0.          9733680.63179705]]

Minimum energy is  -135378.91200422644



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):5878999.568167443, ('q2','q2'):8801064.43183256, ('q3','q3'):11237839.36820295, ('q4','q4'):9733680.63179705}

quadratic = {('q1','q3'):7269966.2362297, ('q1','q4'):-7269966.23

In [80]:
linear = {('q1','q1'):5878999.568167443, ('q2','q2'):8801064.43183256, ('q3','q3'):11237839.36820295, ('q4','q4'):9733680.63179705}

quadratic = {('q1','q3'):7269966.2362297, ('q1','q4'):-7269966.2362297, ('q2','q3'):-7269966.2362297, ('q2','q4'):7269966.2362297} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [81]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  9
Solution x is 3.21699087727594815078E+03
Current  x is 3.07200000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 1.44990877275948150782E+02
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ 1104491.78408372        0.          1817491.55905743 -1817491.55905743]
 [       0.          2565524.21591628 -1817491.55905743  1817491.55905743]
 [       0.                0.          2997479.68410148        0.        ]
 [       0.                0.                0.          2245400.31589852]]

Minimum energy is  -135378.91200422644



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1104491.7840837217, ('q2','q2'):2565524.2159162792, ('q3','q3'):2997479.684101477, ('q4','q4'):2245400.3158985237}

quadratic = {('q1','q3'):1817491.559057425, ('q1','q4'):-1817

In [82]:
linear = {('q1','q1'):1104491.7840837217, ('q2','q2'):2565524.2159162792, ('q3','q3'):2997479.684101477, ('q4','q4'):2245400.3158985237}

quadratic = {('q1','q3'):1817491.559057425, ('q1','q4'):-1817491.559057425, ('q2','q3'):-1817491.559057425, ('q2','q4'):1817491.559057425} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4         energy num_oc. chain_.
0  0  0  0  0            0.0     998     0.0
1  1  0  0  0 1104491.784084       2     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [83]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  8
Solution x is 3.21699087727594815078E+03
Current  x is 3.07200000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 1.44990877275948150782E+02
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[  93493.89204186       0.          454372.88976436 -454372.88976436]
 [      0.          824010.10795814 -454372.88976436  454372.88976436]
 [      0.               0.          843379.84205074       0.        ]
 [      0.               0.               0.          467340.15794926]]

Minimum energy is  -135378.91200422644



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):93493.89204186074, ('q2','q2'):824010.1079581395, ('q3','q3'):843379.8420507382, ('q4','q4'):467340.1579492619}

quadratic = {('q1','q3'):454372.88976435625, ('q1','q4'):-454372.88976435625, ('

In [84]:
linear = {('q1','q1'):93493.89204186074, ('q2','q2'):824010.1079581395, ('q3','q3'):843379.8420507382, ('q4','q4'):467340.1579492619}

quadratic = {('q1','q3'):454372.88976435625, ('q1','q4'):-454372.88976435625, ('q2','q3'):-454372.88976435625, ('q2','q4'):454372.88976435625} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4        energy num_oc. chain_.
0  0  0  0  0           0.0     933     0.0
1  1  0  0  0  93493.892042      38     0.0
2  1  0  0  1 106461.160227      29     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [85]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  7
Solution x is 3.21699087727594815078E+03
Current  x is 3.07200000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is 0.00000000000000000000E+00
Previous error of x is 1.44990877275948150782E+02
Previous error of y is -8.69850185106894429055E+01


# Matrix Q is
[[ -67941.05397907       0.          113593.22244109 -113593.22244109]
 [      0.          297317.05397907 -113593.22244109  113593.22244109]
 [      0.               0.          257849.92102537       0.        ]
 [      0.               0.               0.           69830.07897463]]

Minimum energy is  -135378.91200422644



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-67941.05397906966, ('q2','q2'):297317.0539790697, ('q3','q3'):257849.92102536905, ('q4','q4'):69830.07897463095}

quadratic = {('q1','q3'):113593.22244108906, ('q1','q4'):-113593.22244108906, 

In [86]:
linear = {('q1','q1'):-67941.05397906966, ('q2','q2'):297317.0539790697, ('q3','q3'):257849.92102536905, ('q4','q4'):69830.07897463095}

quadratic = {('q1','q3'):113593.22244108906, ('q1','q4'):-113593.22244108906, ('q2','q3'):-113593.22244108906, ('q2','q4'):113593.22244108906} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4         energy num_oc. chain_.
0  1  0  0  1 -111704.197446     997     0.0
1  1  0  0  0  -67941.053979       3     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [87]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  7
Solution x is 3.21699087727594815078E+03
Current  x is 3.20000000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -1.28000000000000000000E+02
Previous error of x is 1.69908772759481507819E+01
Previous error of y is 4.10149814893105570945E+01


# Matrix Q is
[[  47841.72357984       0.          113593.22244109 -113593.22244109]
 [      0.          181534.27642016 -113593.22244109  113593.22244109]
 [      0.               0.           43763.14346646       0.        ]
 [      0.               0.               0.          283916.85653354]]

Minimum energy is  -23674.71455869886



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):47841.72357984065, ('q2','q2'):181534.2764201594, ('q3','q3'):43763.143466457856, ('q4','q4'):283916.85653354216}

quadratic = {('q1','q3'):113593.22244108906, ('q1','q4'):-113593.22244108906, (

In [88]:
linear = {('q1','q1'):47841.72357984065, ('q2','q2'):181534.2764201594, ('q3','q3'):43763.143466457856, ('q4','q4'):283916.85653354216}

quadratic = {('q1','q3'):113593.22244108906, ('q1','q4'):-113593.22244108906, ('q2','q3'):-113593.22244108906, ('q2','q4'):113593.22244108906} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4       energy num_oc. chain_.
0  0  0  0  0          0.0     994     0.0
1  0  0  1  0 43763.143466       6     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [89]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  6
Solution x is 3.21699087727594815078E+03
Current  x is 3.20000000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -1.28000000000000000000E+02
Previous error of x is 1.69908772759481507819E+01
Previous error of y is 4.10149814893105570945E+01


# Matrix Q is
[[ -4751.13821008      0.          28398.30561027 -28398.30561027]
 [     0.          62095.13821008 -28398.30561027  28398.30561027]
 [     0.              0.         -19078.42826677      0.        ]
 [     0.              0.              0.         100998.42826677]]

Minimum energy is  -23674.71455869886



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-4751.138210079691, ('q2','q2'):62095.138210079705, ('q3','q3'):-19078.42826677107, ('q4','q4'):100998.42826677108}

quadratic = {('q1','q3'):28398.305610272266, ('q1','q4'):-28398.305610272266, ('q2','q3'):-28

In [90]:
linear = {('q1','q1'):-4751.138210079691, ('q2','q2'):62095.138210079705, ('q3','q3'):-19078.42826677107, ('q4','q4'):100998.42826677108}

quadratic = {('q1','q3'):28398.305610272266, ('q1','q4'):-28398.305610272266, ('q2','q3'):-28398.305610272266, ('q2','q4'):28398.305610272266} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4        energy num_oc. chain_.
0  0  0  1  0 -19078.428267     994     0.0
1  1  0  0  0   -4751.13821       5     0.0
2  1  0  1  0   4568.739133       1     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [91]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  6
Solution x is 3.21699087727594815078E+03
Current  x is 3.20000000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -6.40000000000000000000E+01
Previous error of x is 1.69908772759481507819E+01
Previous error of y is -2.29850185106894429055E+01


# Matrix Q is
[[ 23647.16740019      0.          28398.30561027 -28398.30561027]
 [     0.          33696.83259981 -28398.30561027  28398.30561027]
 [     0.              0.          62841.57173323      0.        ]
 [     0.              0.              0.          19078.42826677]]

Minimum energy is  -4596.286291927743



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):23647.167400192862, ('q2','q2'):33696.83259980715, ('q3','q3'):62841.571733229015, ('q4','q4'):19078.428266770992}

quadratic = {('q1','q3'):28398.305610272266, ('q1','q4'):-28398.305610272266, ('q2','q3'):-28

In [92]:
linear = {('q1','q1'):23647.167400192862, ('q2','q2'):33696.83259980715, ('q3','q3'):62841.571733229015, ('q4','q4'):19078.428266770992}

quadratic = {('q1','q3'):28398.305610272266, ('q1','q4'):-28398.305610272266, ('q2','q3'):-28398.305610272266, ('q2','q4'):28398.305610272266} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4       energy num_oc. chain_.
0  0  0  0  0          0.0     991     0.0
1  1  0  0  1 14327.290057       9     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [93]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  5
Solution x is 3.21699087727594815078E+03
Current  x is 3.20000000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -6.40000000000000000000E+01
Previous error of x is 1.69908772759481507819E+01
Previous error of y is -2.29850185106894429055E+01


# Matrix Q is
[[ 4655.5837001      0.          7099.57640257 -7099.57640257]
 [    0.          9680.4162999  -7099.57640257  7099.57640257]
 [    0.             0.         21180.78586661     0.        ]
 [    0.             0.             0.          -700.78586661]]

Minimum energy is  -4596.286291927743



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):4655.583700096429, ('q2','q2'):9680.416299903573, ('q3','q3'):21180.785866614508, ('q4','q4'):-700.7858666145053}

quadratic = {('q1','q3'):7099.576402568066, ('q1','q4'):-7099.576402568066, ('q2','q3'):-7099.576402568066, ('

In [94]:
linear = {('q1','q1'):4655.583700096429, ('q2','q2'):9680.416299903573, ('q3','q3'):21180.785866614508, ('q4','q4'):-700.7858666145053}

quadratic = {('q1','q3'):7099.576402568066, ('q1','q4'):-7099.576402568066, ('q2','q3'):-7099.576402568066, ('q2','q4'):7099.576402568066} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4       energy num_oc. chain_.
0  1  0  0  1 -3144.778569     991     0.0
1  0  0  0  1  -700.785867       6     0.0
2  0  0  0  0          0.0       3     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [95]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  5
Solution x is 3.21699087727594815078E+03
Current  x is 3.23200000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -9.60000000000000000000E+01
Previous error of x is -1.50091227240518492181E+01
Previous error of y is 9.01498148931055709454E+00


# Matrix Q is
[[11892.00729753     0.          7099.57640257 -7099.57640257]
 [    0.          2443.99270247 -7099.57640257  7099.57640257]
 [    0.             0.          7800.36226918     0.        ]
 [    0.             0.             0.         12679.63773082]]

Minimum energy is  -1451.507722841606



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):11892.007297528377, ('q2','q2'):2443.992702471627, ('q3','q3'):7800.362269182569, ('q4','q4'):12679.637730817432}

quadratic = {('q1','q3'):7099.576402568066, ('q1','q4'):-7099.576402568066, ('q2','q3'):-7099.576402568066, ('

In [96]:
linear = {('q1','q1'):11892.007297528377, ('q2','q2'):2443.992702471627, ('q3','q3'):7800.362269182569, ('q4','q4'):12679.637730817432}

quadratic = {('q1','q3'):7099.576402568066, ('q1','q4'):-7099.576402568066, ('q2','q3'):-7099.576402568066, ('q2','q4'):7099.576402568066} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4      energy num_oc. chain_.
0  0  0  0  0         0.0     995     0.0
1  0  1  0  0 2443.992702       3     0.0
2  0  1  1  0 3144.778569       2     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [97]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  4
Solution x is 3.21699087727594815078E+03
Current  x is 3.23200000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -9.60000000000000000000E+01
Previous error of x is -1.50091227240518492181E+01
Previous error of y is 9.01498148931055709454E+00


# Matrix Q is
[[ 4154.00364876     0.          1774.89410064 -1774.89410064]
 [    0.          -570.00364876 -1774.89410064  1774.89410064]
 [    0.             0.          1340.18113459     0.        ]
 [    0.             0.             0.          3779.81886541]]

Minimum energy is  -1451.507722841606



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):4154.003648764187, ('q2','q2'):-570.0036487641866, ('q3','q3'):1340.1811345912845, ('q4','q4'):3779.8188654087153}

quadratic = {('q1','q3'):1774.8941006420166, ('q1','q4'):-1774.8941006420166, ('q2','q3'):-1774.8941006420166

In [98]:
linear = {('q1','q1'):4154.003648764187, ('q2','q2'):-570.0036487641866, ('q3','q3'):1340.1811345912845, ('q4','q4'):3779.8188654087153}

quadratic = {('q1','q3'):1774.8941006420166, ('q1','q4'):-1774.8941006420166, ('q2','q3'):-1774.8941006420166, ('q2','q4'):1774.8941006420166} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4       energy num_oc. chain_.
0  0  1  1  0 -1004.716615     993     0.0
1  0  1  0  0  -570.003649       7     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [99]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  4
Solution x is 3.21699087727594815078E+03
Current  x is 3.21600000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.00000000000000000000E+01
Previous error of x is 9.90877275948150781915E-01
Previous error of y is -6.98501851068944290546E+00


# Matrix Q is
[[ 2344.89774941     0.          1774.89410064 -1774.89410064]
 [    0.          1239.10225059 -1774.89410064  1774.89410064]
 [    0.             0.          4685.28703395     0.        ]
 [    0.             0.             0.           434.71296605]]

Minimum energy is  -446.7911080267147



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):2344.897749406213, ('q2','q2'):1239.102250593788, ('q3','q3'):4685.287033949333, ('q4','q4'):434.712966050668}

quadratic = {('q1','q3'):1774.8941006420166, ('q1','q4'):-1774.8941006420166, ('q2','q3'):-1774.8941006420166, ('

In [100]:
linear = {('q1','q1'):2344.897749406213, ('q2','q2'):1239.102250593788, ('q3','q3'):4685.287033949333, ('q4','q4'):434.712966050668}

quadratic = {('q1','q3'):1774.8941006420166, ('q1','q4'):-1774.8941006420166, ('q2','q3'):-1774.8941006420166, ('q2','q4'):1774.8941006420166} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4      energy num_oc. chain_.
0  0  0  0  0         0.0     972     0.0
1  0  0  0  1  434.712966      25     0.0
2  1  0  0  1 1004.716615       3     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [101]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  3
Solution x is 3.21699087727594815078E+03
Current  x is 3.21600000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.00000000000000000000E+01
Previous error of x is 9.90877275948150781915E-01
Previous error of y is -6.98501851068944290546E+00


# Matrix Q is
[[ 724.4488747     0.          443.72352516 -443.72352516]
 [   0.          171.5511253  -443.72352516  443.72352516]
 [   0.            0.         1702.64351697    0.        ]
 [   0.            0.            0.         -422.64351697]]

Minimum energy is  -446.7911080267147



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):724.4488747031063, ('q2','q2'):171.55112529689382, ('q3','q3'):1702.6435169746662, ('q4','q4'):-422.6435169746661}

quadratic = {('q1','q3'):443.72352516050415, ('q1','q4'):-443.72352516050415, ('q2','q3'):-443.72352516050415, ('q2','q4'):44

In [102]:
linear = {('q1','q1'):724.4488747031063, ('q2','q2'):171.55112529689382, ('q3','q3'):1702.6435169746662, ('q4','q4'):-422.6435169746661}

quadratic = {('q1','q3'):443.72352516050415, ('q1','q4'):-443.72352516050415, ('q2','q3'):-443.72352516050415, ('q2','q4'):443.72352516050415} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4      energy num_oc. chain_.
0  0  0  0  1 -422.643517     995     0.0
1  1  0  0  1 -141.918167       5     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [103]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  3
Solution x is 3.21699087727594815078E+03
Current  x is 3.21600000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.80000000000000000000E+01
Previous error of x is 9.90877275948150781915E-01
Previous error of y is 1.01498148931055709454E+00


# Matrix Q is
[[ 280.72534954    0.          443.72352516 -443.72352516]
 [   0.          615.27465046 -443.72352516  443.72352516]
 [   0.            0.          422.64351697    0.        ]
 [   0.            0.            0.          857.35648303]]

Minimum energy is  -24.147591052049766



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):280.7253495426131, ('q2','q2'):615.274650457387, ('q3','q3'):422.64351697464605, ('q4','q4'):857.356483025354}

quadratic = {('q1','q3'):443.72352516050415, ('q1','q4'):-443.72352516050415, ('q2','q3'):-443.72352516050415, ('q2','q4'):443.72

In [104]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):280.7253495426131, ('q2','q2'):615.274650457387, ('q3','q3'):422.64351697464605, ('q4','q4'):857.356483025354}

quadratic = {('q1','q3'):443.72352516050415, ('q1','q4'):-443.72352516050415, ('q2','q3'):-443.72352516050415, ('q2','q4'):443.72352516050415} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4     energy num_oc. chain_.
0  0  0  0  0        0.0     999     0.0
1  0  0  0  1 857.356483       1     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [105]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  2
Solution x is 3.21699087727594815078E+03
Current  x is 3.21600000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.80000000000000000000E+01
Previous error of x is 9.90877275948150781915E-01
Previous error of y is 1.01498148931055709454E+00


# Matrix Q is
[[  28.36267477    0.          110.93088129 -110.93088129]
 [   0.          195.63732523 -110.93088129  110.93088129]
 [   0.            0.           51.32175849    0.        ]
 [   0.            0.            0.          268.67824151]]

Minimum energy is  -24.147591052049766



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):28.362674771306544, ('q2','q2'):195.6373252286935, ('q3','q3'):51.321758487322995, ('q4','q4'):268.678241512677}

quadratic = {('q1','q3'):110.93088129012604, ('q1','q4'):-110.93088129012604, ('q2','q3'):-110.93088129012604, ('q2','q4'):110.

In [106]:
linear = {('q1','q1'):28.362674771306544, ('q2','q2'):195.6373252286935, ('q3','q3'):51.321758487322995, ('q4','q4'):268.678241512677}

quadratic = {('q1','q3'):110.93088129012604, ('q1','q4'):-110.93088129012604, ('q2','q3'):-110.93088129012604, ('q2','q4'):110.93088129012604} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4    energy num_oc. chain_.
0  0  0  0  0       0.0     967     0.0
1  1  0  0  0 28.362675      32     0.0
2  0  0  1  0 51.321758       1     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [107]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  1
Solution x is 3.21699087727594815078E+03
Current  x is 3.21600000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.80000000000000000000E+01
Previous error of x is 9.90877275948150781915E-01
Previous error of y is 1.01498148931055709454E+00


# Matrix Q is
[[-13.81866261   0.          27.73272032 -27.73272032]
 [  0.          69.81866261 -27.73272032  27.73272032]
 [  0.           0.         -14.33912076   0.        ]
 [  0.           0.           0.          94.33912076]]

Minimum energy is  -24.147591052049766



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-13.818662614346728, ('q2','q2'):69.81866261434674, ('q3','q3'):-14.339120756338502, ('q4','q4'):94.3391207563385}

quadratic = {('q1','q3'):27.73272032253151, ('q1','q4'):-27.73272032253151, ('q2','q3'):-27.73272032253151, ('q2','q4'):27.73272032253151} 
Q

In [108]:
linear = {('q1','q1'):-13.818662614346728, ('q2','q2'):69.81866261434674, ('q3','q3'):-14.339120756338502, ('q4','q4'):94.3391207563385}

quadratic = {('q1','q3'):27.73272032253151, ('q1','q4'):-27.73272032253151, ('q2','q3'):-27.73272032253151, ('q2','q4'):27.73272032253151} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4     energy num_oc. chain_.
0  0  0  1  0 -14.339121     694     0.0
1  1  0  0  0 -13.818663     303     0.0
2  1  0  1  0  -0.425063       2     0.0
3  0  0  0  0        0.0       1     0.0
['BINARY', 4 rows, 1000 samples, 4 variables]


In [109]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  1
Solution x is 3.21699087727594815078E+03
Current  x is 3.21600000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.60000000000000000000E+01
Previous error of x is 9.90877275948150781915E-01
Previous error of y is -9.85018510689442905459E-01


# Matrix Q is
[[ 13.91405771   0.          27.73272032 -27.73272032]
 [  0.          42.08594229 -27.73272032  27.73272032]
 [  0.           0.          65.66087924   0.        ]
 [  0.           0.           0.          14.33912076]]

Minimum energy is  -9.808470295713455



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):13.914057708180778, ('q2','q2'):42.08594229181923, ('q3','q3'):65.66087924366192, ('q4','q4'):14.339120756338076}

quadratic = {('q1','q3'):27.73272032253151, ('q1','q4'):-27.73272032253151, ('q2','q3'):-27.73272032253151, ('q2','q4'):27.73272032253151} 
Q 

In [110]:
linear = {('q1','q1'):13.914057708180778, ('q2','q2'):42.08594229181923, ('q3','q3'):65.66087924366192, ('q4','q4'):14.339120756338076}

quadratic = {('q1','q3'):27.73272032253151, ('q1','q4'):-27.73272032253151, ('q2','q3'):-27.73272032253151, ('q2','q4'):27.73272032253151} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4    energy num_oc. chain_.
0  0  0  0  0       0.0     363     0.0
1  1  0  0  1  0.520458     635     0.0
2  1  0  0  0 13.914058       2     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [111]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  0
Solution x is 3.21699087727594815078E+03
Current  x is 3.21600000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.60000000000000000000E+01
Previous error of x is 9.90877275948150781915E-01
Previous error of y is -9.85018510689442905459E-01


# Matrix Q is
[[-0.04297115  0.          6.93318008 -6.93318008]
 [ 0.         14.04297115 -6.93318008  6.93318008]
 [ 0.          0.         22.83043962  0.        ]
 [ 0.          0.          0.         -2.83043962]]

Minimum energy is  -9.808470295713455



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-0.04297114590961337, ('q2','q2'):14.042971145909615, ('q3','q3'):22.830439621830962, ('q4','q4'):-2.830439621830961}

quadratic = {('q1','q3'):6.933180080632877, ('q1','q4'):-6.933180080632877, ('q2','q3'):-6.933180080632877, ('q2','q4'):6.933180080632877} 
Q = dict(linea

In [112]:
linear = {('q1','q1'):-0.04297114590961337, ('q2','q2'):14.042971145909615, ('q3','q3'):22.830439621830962, ('q4','q4'):-2.830439621830961}

quadratic = {('q1','q3'):6.933180080632877, ('q1','q4'):-6.933180080632877, ('q2','q3'):-6.933180080632877, ('q2','q4'):6.933180080632877} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4    energy num_oc. chain_.
0  1  0  0  1 -9.806591    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [113]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  0
Solution x is 3.21699087727594815078E+03
Current  x is 3.21700000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.70000000000000000000E+01
Previous error of x is -9.12272405184921808541E-03
Previous error of y is 1.49814893105570945409E-02


# Matrix Q is
[[ 7.02384877  0.          6.93318008 -6.93318008]
 [ 0.          6.97615123 -6.93318008  6.93318008]
 [ 0.          0.          9.7636197   0.        ]
 [ 0.          0.          0.         10.2363803 ]]

Minimum energy is  -0.0018794473400537642



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):7.02384877345682, ('q2','q2'):6.976151226543182, ('q3','q3'):9.763619702460105, ('q4','q4'):10.236380297539897}

quadratic = {('q1','q3'):6.933180080632877, ('q1','q4'):-6.933180080632877, ('q2','q3'):-6.933180080632877, ('q2','q4'):6.933180080632877} 
Q = dict(linear)

In [114]:
linear = {('q1','q1'):7.02384877345682, ('q2','q2'):6.976151226543182, ('q3','q3'):9.763619702460105, ('q4','q4'):10.236380297539897}

quadratic = {('q1','q3'):6.933180080632877, ('q1','q4'):-6.933180080632877, ('q2','q3'):-6.933180080632877, ('q2','q4'):6.933180080632877} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  0  0  0  0      0.0     998     0.0
1  0  1  0  0 6.976151       1     0.0
2  0  0  0  1 10.23638       1     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [115]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -1
Solution x is 3.21699087727594815078E+03
Current  x is 3.21700000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.70000000000000000000E+01
Previous error of x is -9.12272405184921808541E-03
Previous error of y is 1.49814893105570945409E-02


# Matrix Q is
[[ 1.76192439  0.          1.73329502 -1.73329502]
 [ 0.          1.73807561 -1.73329502  1.73329502]
 [ 0.          0.          2.38180985  0.        ]
 [ 0.          0.          0.          2.61819015]]

Minimum energy is  -0.0018794473400537642



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.76192438672841, ('q2','q2'):1.7380756132715907, ('q3','q3'):2.3818098512300514, ('q4','q4'):2.618190148769949}

quadratic = {('q1','q3'):1.7332950201582193, ('q1','q4'):-1.7332950201582193, ('q2','q3'):-1.7332950201582193, ('q2','q4'):1.7332950201582193} 
Q = dict(l

In [116]:
linear = {('q1','q1'):1.76192438672841, ('q2','q2'):1.7380756132715907, ('q3','q3'):2.3818098512300514, ('q4','q4'):2.618190148769949}

quadratic = {('q1','q3'):1.7332950201582193, ('q1','q4'):-1.7332950201582193, ('q2','q3'):-1.7332950201582193, ('q2','q4'):1.7332950201582193} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [117]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -2
Solution x is 3.21699087727594815078E+03
Current  x is 3.21700000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.70000000000000000000E+01
Previous error of x is -9.12272405184921808541E-03
Previous error of y is 1.49814893105570945409E-02


# Matrix Q is
[[ 0.44346219  0.          0.43332376 -0.43332376]
 [ 0.          0.43153781 -0.43332376  0.43332376]
 [ 0.          0.          0.56590493  0.        ]
 [ 0.          0.          0.          0.68409507]]

Minimum energy is  -0.0018794473400537642



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):0.4434621933642049, ('q2','q2'):0.43153780663579533, ('q3','q3'):0.5659049256150257, ('q4','q4'):0.6840950743849744}

quadratic = {('q1','q3'):0.43332375503955484, ('q1','q4'):-0.43332375503955484, ('q2','q3'):-0.43332375503955484, ('q2','q4'):0.43332375503955484} 
Q 

In [118]:
linear = {('q1','q1'):0.4434621933642049, ('q2','q2'):0.43153780663579533, ('q3','q3'):0.5659049256150257, ('q4','q4'):0.6840950743849744}

quadratic = {('q1','q3'):0.43332375503955484, ('q1','q4'):-0.43332375503955484, ('q2','q3'):-0.43332375503955484, ('q2','q4'):0.43332375503955484} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [119]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -3
Solution x is 3.21699087727594815078E+03
Current  x is 3.21700000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.70000000000000000000E+01
Previous error of x is -9.12272405184921808541E-03
Previous error of y is 1.49814893105570945409E-02


# Matrix Q is
[[ 0.1123561   0.          0.10833094 -0.10833094]
 [ 0.          0.1063939  -0.10833094  0.10833094]
 [ 0.          0.          0.12670246  0.        ]
 [ 0.          0.          0.          0.18579754]]

Minimum energy is  -0.0018794473400537642



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):0.11235609668210242, ('q2','q2'):0.10639390331789762, ('q3','q3'):0.12670246280751282, ('q4','q4'):0.18579753719248718}

quadratic = {('q1','q3'):0.10833093875988871, ('q1','q4'):-0.10833093875988871, ('q2','q3'):-0.10833093875988871, ('q2','q4'):0.10833093875988871} 

In [120]:
linear = {('q1','q1'):0.11235609668210242, ('q2','q2'):0.10639390331789762, ('q3','q3'):0.12670246280751282, ('q4','q4'):0.18579753719248718}

quadratic = {('q1','q3'):0.10833093875988871, ('q1','q4'):-0.10833093875988871, ('q2','q3'):-0.10833093875988871, ('q2','q4'):0.10833093875988871} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [121]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -4
Solution x is 3.21699087727594815078E+03
Current  x is 3.21700000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.70000000000000000000E+01
Previous error of x is -9.12272405184921808541E-03
Previous error of y is 1.49814893105570945409E-02


# Matrix Q is
[[ 0.0288343   0.          0.02708273 -0.02708273]
 [ 0.          0.0258532  -0.02708273  0.02708273]
 [ 0.          0.          0.02428873  0.        ]
 [ 0.          0.          0.          0.05383627]]

Minimum energy is  -0.0018794473400537642



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):0.028834298341051202, ('q2','q2'):0.025853201658948808, ('q3','q3'):0.024288731403756418, ('q4','q4'):0.05383626859624359}

quadratic = {('q1','q3'):0.027082734689972177, ('q1','q4'):-0.027082734689972177, ('q2','q3'):-0.027082734689972177, ('q2','q4'):0.0270827346899

In [122]:
linear = {('q1','q1'):0.028834298341051202, ('q2','q2'):0.025853201658948808, ('q3','q3'):0.024288731403756418, ('q4','q4'):0.05383626859624359}

quadratic = {('q1','q3'):0.027082734689972177, ('q1','q4'):-0.027082734689972177, ('q2','q3'):-0.027082734689972177, ('q2','q4'):0.027082734689972177} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [123]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -5
Solution x is 3.21699087727594815078E+03
Current  x is 3.21700000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.70000000000000000000E+01
Previous error of x is -9.12272405184921808541E-03
Previous error of y is 1.49814893105570945409E-02


# Matrix Q is
[[ 0.00758121  0.          0.00677068 -0.00677068]
 [ 0.          0.00609066 -0.00677068  0.00677068]
 [ 0.          0.          0.00237874  0.        ]
 [ 0.          0.          0.          0.01715251]]

Minimum energy is  -0.0018794473400537642



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):0.007581211670525601, ('q2','q2'):0.006090663329474402, ('q3','q3'):0.002378740701878209, ('q4','q4'):0.017152509298121794}

quadratic = {('q1','q3'):0.006770683672493044, ('q1','q4'):-0.006770683672493044, ('q2','q3'):-0.006770683672493044, ('q2','q4'):0.006770683672

In [125]:
linear = {('q1','q1'):0.007581211670525601, ('q2','q2'):0.006090663329474402, ('q3','q3'):0.002378740701878209, ('q4','q4'):0.017152509298121794}

quadratic = {('q1','q3'):0.006770683672493044, ('q1','q4'):-0.006770683672493044, ('q2','q3'):-0.006770683672493044, ('q2','q4'):0.006770683672493044} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  0  0  0  0      0.0     887     0.0
1  0  1  1  0 0.001699      96     0.0
2  0  0  1  0 0.002379      17     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [126]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -6
Solution x is 3.21699087727594815078E+03
Current  x is 3.21700000000000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.70000000000000000000E+01
Previous error of x is -9.12272405184921808541E-03
Previous error of y is 1.49814893105570945409E-02


# Matrix Q is
[[ 0.00208162  0.          0.00169267 -0.00169267]
 [ 0.          0.00133635 -0.00169267  0.00169267]
 [ 0.          0.         -0.00125204  0.        ]
 [ 0.          0.          0.          0.00613485]]

Minimum energy is  -0.0018794473400537642



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):0.0020816214602627997, ('q2','q2'):0.001336347289737201, ('q3','q3'):-0.0012520358990608957, ('q4','q4'):0.006134848399060896}

quadratic = {('q1','q3'):0.001692670918123261, ('q1','q4'):-0.001692670918123261, ('q2','q3'):-0.001692670918123261, ('q2','q4'):0.001692670

In [127]:
linear = {('q1','q1'):0.0020816214602627997, ('q2','q2'):0.001336347289737201, ('q3','q3'):-0.0012520358990608957, ('q4','q4'):0.006134848399060896}

quadratic = {('q1','q3'):0.001692670918123261, ('q1','q4'):-0.001692670918123261, ('q2','q3'):-0.001692670918123261, ('q2','q4'):0.001692670918123261} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4    energy num_oc. chain_.
0  0  1  1  0 -0.001608     911     0.0
1  0  0  1  0 -0.001252      88     0.0
2  0  0  0  0       0.0       1     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [128]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -6
Solution x is 3.21699087727594815078E+03
Current  x is 3.21698437500000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69843750000000000000E+01
Previous error of x is 6.50227594815078191459E-03
Previous error of y is -6.43510689442905459146E-04


# Matrix Q is
[[ 0.00035632  0.          0.00169267 -0.00169267]
 [ 0.          0.00306165 -0.00169267  0.00169267]
 [ 0.          0.          0.00193811  0.        ]
 [ 0.          0.          0.          0.00294471]]

Minimum energy is  -0.00027108781259751655



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):0.0003563236284126162, ('q2','q2'):0.0030616451215873842, ('q3','q3'):0.0019381056828585632, ('q4','q4'):0.002944706817141437}

quadratic = {('q1','q3'):0.001692670918123261, ('q1','q4'):-0.001692670918123261, ('q2','q3'):-0.001692670918123261, ('q2','q4'):0.00169267

In [129]:
linear = {('q1','q1'):0.0003563236284126162, ('q2','q2'):0.0030616451215873842, ('q3','q3'):0.0019381056828585632, ('q4','q4'):0.002944706817141437}

quadratic = {('q1','q3'):0.001692670918123261, ('q1','q4'):-0.001692670918123261, ('q2','q3'):-0.001692670918123261, ('q2','q4'):0.001692670918123261} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  0  0  0  0      0.0     969     0.0
1  1  0  0  0 0.000356      31     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [130]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -7
Solution x is 3.21699087727594815078E+03
Current  x is 3.21698437500000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69843750000000000000E+01
Previous error of x is 6.50227594815078191459E-03
Previous error of y is -6.43510689442905459146E-04


# Matrix Q is
[[-0.00024908  0.          0.00042317 -0.00042317]
 [ 0.          0.00110358 -0.00042317  0.00042317]
 [ 0.          0.          0.0003587   0.        ]
 [ 0.          0.          0.          0.000862  ]]

Minimum energy is  -0.00027108781259751655



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-0.000249084279543692, ('q2','q2'):0.0011035764670436921, ('q3','q3'):0.0003587012789292816, ('q4','q4'):0.0008620018460707185}

quadratic = {('q1','q3'):0.00042316772953081527, ('q1','q4'):-0.00042316772953081527, ('q2','q3'):-0.00042316772953081527, ('q2','q4'):0.0

In [131]:
linear = {('q1','q1'):-0.000249084279543692, ('q2','q2'):0.0011035764670436921, ('q3','q3'):0.0003587012789292816, ('q4','q4'):0.0008620018460707185}

quadratic = {('q1','q3'):0.00042316772953081527, ('q1','q4'):-0.00042316772953081527, ('q2','q3'):-0.00042316772953081527, ('q2','q4'):0.00042316772953081527} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4    energy num_oc. chain_.
0  1  0  0  0 -0.000249     999     0.0
1  0  0  0  0       0.0       1     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [132]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -7
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699218750000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69843750000000000000E+01
Previous error of x is -1.31022405184921808541E-03
Previous error of y is -6.43510689442905459146E-04


# Matrix Q is
[[ 0.00060541  0.          0.00042317 -0.00042317]
 [ 0.          0.00024908 -0.00042317  0.00042317]
 [ 0.          0.          0.00078187  0.        ]
 [ 0.          0.          0.          0.00043883]]

Minimum energy is  -2.2003533054046523e-05



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):0.0006054079079585855, ('q2','q2'):0.00024908427954141476, ('q3','q3'):0.0007818690084581553, ('q4','q4'):0.0004388341165418448}

quadratic = {('q1','q3'):0.00042316772953081527, ('q1','q4'):-0.00042316772953081527, ('q2','q3'):-0.00042316772953081527, ('q2','q4'):0

In [133]:
linear = {('q1','q1'):0.0006054079079585855, ('q2','q2'):0.00024908427954141476, ('q3','q3'):0.0007818690084581553, ('q4','q4'):0.0004388341165418448}

quadratic = {('q1','q3'):0.00042316772953081527, ('q1','q4'):-0.00042316772953081527, ('q2','q3'):-0.00042316772953081527, ('q2','q4'):0.00042316772953081527} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [134]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -8
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699218750000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69843750000000000000E+01
Previous error of x is -1.31022405184921808541E-03
Previous error of y is -6.43510689442905459146E-04


# Matrix Q is
[[ 1.95892431e-04  0.00000000e+00  1.05791932e-04 -1.05791932e-04]
 [ 0.00000000e+00  1.77306163e-05 -1.05791932e-04  1.05791932e-04]
 [ 0.00000000e+00  0.00000000e+00  2.38346614e-04  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.68291676e-05]]

Minimum energy is  -2.2003533054046523e-05



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):0.0001958924305417927, ('q2','q2'):1.773061633320734e-05, ('q3','q3'):0.0002383466136040776, ('q4','q4'):6.68291676459224e-05}

quadratic = {('q1','q3'):0.00010579193238270382, ('q1','q4'):-0.0001057919

In [135]:
linear = {('q1','q1'):0.0001958924305417927, ('q2','q2'):1.773061633320734e-05, ('q3','q3'):0.0002383466136040776, ('q4','q4'):6.68291676459224e-05}

quadratic = {('q1','q3'):0.00010579193238270382, ('q1','q4'):-0.00010579193238270382, ('q2','q3'):-0.00010579193238270382, ('q2','q4'):0.00010579193238270382} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  0  0  0  0      0.0     952     0.0
1  0  1  0  0 0.000018      45     0.0
2  0  0  0  1 0.000067       3     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [136]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -9
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699218750000000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69843750000000000000E+01
Previous error of x is -1.31022405184921808541E-03
Previous error of y is -6.43510689442905459146E-04


# Matrix Q is
[[ 7.12433344e-05  0.00000000e+00  2.64479831e-05 -2.64479831e-05]
 [ 0.00000000e+00 -1.78375727e-05 -2.64479831e-05  2.64479831e-05]
 [ 0.00000000e+00  0.00000000e+00  8.10263341e-05  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.73238883e-06]]

Minimum energy is  -2.2003533054046523e-05



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):7.124333441152134e-05, ('q2','q2'):-1.7837572692771334e-05, ('q3','q3'):8.10263341457888e-05, ('q4','q4'):-4.732388833288797e-06}

quadratic = {('q1','q3'):2.6447983095675954e-05, ('q1','q4'):-2.6447983

In [137]:
linear = {('q1','q1'):7.124333441152134e-05, ('q2','q2'):-1.7837572692771334e-05, ('q3','q3'):8.10263341457888e-05, ('q4','q4'):-4.732388833288797e-06}

quadratic = {('q1','q3'):2.6447983095675954e-05, ('q1','q4'):-2.6447983095675954e-05, ('q2','q3'):-2.6447983095675954e-05, ('q2','q4'):2.6447983095675954e-05} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4    energy num_oc. chain_.
0  0  1  0  0 -0.000018     978     0.0
1  0  0  0  1 -0.000005      14     0.0
2  0  0  0  0       0.0       3     0.0
3  0  1  0  1  0.000004       3     0.0
4  0  1  1  0  0.000037       1     0.0
5  1  1  0  0  0.000053       1     0.0
['BINARY', 6 rows, 1000 samples, 4 variables]


In [138]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -9
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699023437500000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69843750000000000000E+01
Previous error of x is 6.42900948150781914592E-04
Previous error of y is -6.43510689442905459146E-04


# Matrix Q is
[[ 1.78375727e-05  0.00000000e+00  2.64479831e-05 -2.64479831e-05]
 [ 0.00000000e+00  3.55681890e-05 -2.64479831e-05  2.64479831e-05]
 [ 0.00000000e+00  0.00000000e+00  5.45783511e-05  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.17155943e-05]]

Minimum energy is  -4.165960361603036e-06



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.7837572696601058e-05, ('q2','q2'):3.5568189022148955e-05, ('q3','q3'):5.4578351054934e-05, ('q4','q4'):2.1715594257566005e-05}

quadratic = {('q1','q3'):2.6447983095675954e-05, ('q1','q4'):-2.6447983095

In [139]:
linear = {('q1','q1'):1.7837572696601058e-05, ('q2','q2'):3.5568189022148955e-05, ('q3','q3'):5.4578351054934e-05, ('q4','q4'):2.1715594257566005e-05}

quadratic = {('q1','q3'):2.6447983095675954e-05, ('q1','q4'):-2.6447983095675954e-05, ('q2','q3'):-2.6447983095675954e-05, ('q2','q4'):2.6447983095675954e-05} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  0  0  0  0      0.0     995     0.0
1  1  0  0  1 0.000013       5     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [140]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -10
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699023437500000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69843750000000000000E+01
Previous error of x is 6.42900948150781914592E-04
Previous error of y is -6.43510689442905459146E-04


# Matrix Q is
[[ 2.24306613e-06  0.00000000e+00  6.61199577e-06 -6.61199577e-06]
 [ 0.00000000e+00  1.11083743e-05 -6.61199577e-06  6.61199577e-06]
 [ 0.00000000e+00  0.00000000e+00  1.77524324e-05  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.32105396e-06]]

Minimum energy is  -4.165960361603036e-06



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):2.243066133456777e-06, ('q2','q2'):1.1108374296230724e-05, ('q3','q3'):1.7752432363404497e-05, ('q4','q4'):1.3210539647205036e-06}

quadratic = {('q1','q3'):6.611995773918989e-06, ('q1','q4'):-6.61199577

In [141]:
linear = {('q1','q1'):2.243066133456777e-06, ('q2','q2'):1.1108374296230724e-05, ('q3','q3'):1.7752432363404497e-05, ('q4','q4'):1.3210539647205036e-06}

quadratic = {('q1','q3'):6.611995773918989e-06, ('q1','q4'):-6.611995773918989e-06, ('q2','q3'):-6.611995773918989e-06, ('q2','q4'):6.611995773918989e-06} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4    energy num_oc. chain_.
0  1  0  0  1 -0.000003     980     0.0
1  0  0  0  0       0.0      18     0.0
2  0  0  0  1  0.000001       2     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [142]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -10
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699121093750000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69853515625000000000E+01
Previous error of x is -3.33661551849218085408E-04
Previous error of y is 3.33051810557094540854E-04


# Matrix Q is
[[ 8.98251079e-06  0.00000000e+00  6.61199577e-06 -6.61199577e-06]
 [ 0.00000000e+00  4.36892964e-06 -6.61199577e-06  6.61199577e-06]
 [ 0.00000000e+00  0.00000000e+00  5.29094181e-06  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.37825445e-05]]

Minimum energy is  -1.118084685198148e-06



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):8.982510787682993e-06, ('q2','q2'):4.368929642004508e-06, ('q3','q3'):5.290941809597915e-06, ('q4','q4'):1.3782544518527086e-05}

quadratic = {('q1','q3'):6.611995773918989e-06, ('q1','q4'):-6.6119957739

In [143]:
linear = {('q1','q1'):8.982510787682993e-06, ('q2','q2'):4.368929642004508e-06, ('q3','q3'):5.290941809597915e-06, ('q4','q4'):1.3782544518527086e-05}

quadratic = {('q1','q3'):6.611995773918989e-06, ('q1','q4'):-6.611995773918989e-06, ('q2','q3'):-6.611995773918989e-06, ('q2','q4'):6.611995773918989e-06} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  0  0  0  0      0.0     984     0.0
1  0  1  1  0 0.000003      16     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [144]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -11
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699121093750000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69853515625000000000E+01
Previous error of x is -3.33661551849218085408E-04
Previous error of y is 3.33051810557094540854E-04


# Matrix Q is
[[ 2.82232534e-06  0.00000000e+00  1.65299894e-06 -1.65299894e-06]
 [ 0.00000000e+00  5.15534767e-07 -1.65299894e-06  1.65299894e-06]
 [ 0.00000000e+00  0.00000000e+00  2.61285114e-07  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.50708647e-06]]

Minimum energy is  -1.118084685198148e-06



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):2.822325340130559e-06, ('q2','q2'):5.155347672913166e-07, ('q3','q3'):2.6128511378333224e-07, ('q4','q4'):4.507086468247918e-06}

quadratic = {('q1','q3'):1.6529989434797471e-06, ('q1','q4'):-1.652998943

In [145]:
linear = {('q1','q1'):2.822325340130559e-06, ('q2','q2'):5.155347672913166e-07, ('q3','q3'):2.6128511378333224e-07, ('q4','q4'):4.507086468247918e-06}

quadratic = {('q1','q3'):1.6529989434797471e-06, ('q1','q4'):-1.6529989434797471e-06, ('q2','q3'):-1.6529989434797471e-06, ('q2','q4'):1.6529989434797471e-06} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4    energy num_oc. chain_.
0  0  1  1  0 -0.000001     998     0.0
1  0  0  0  0       0.0       1     0.0
2  0  1  0  0  0.000001       1     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [146]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -11
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699072265625000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69848632812500000000E+01
Previous error of x is 1.54619698150781914592E-04
Previous error of y is -1.55229439442905459146E-04


# Matrix Q is
[[ 1.13746417e-06  0.00000000e+00  1.65299894e-06 -1.65299894e-06]
 [ 0.00000000e+00  2.20039593e-06 -1.65299894e-06  1.65299894e-06]
 [ 0.00000000e+00  0.00000000e+00  3.37665775e-06  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.39171383e-06]]

Minimum energy is  -2.4190562296445567e-07



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.1374641739599406e-06, ('q2','q2'):2.200395933461935e-06, ('q3','q3'):3.376657750654264e-06, ('q4','q4'):1.3917138313769865e-06}

quadratic = {('q1','q3'):1.6529989434797471e-06, ('q1','q4'):-1.6529989

In [147]:
linear = {('q1','q1'):1.1374641739599406e-06, ('q2','q2'):2.200395933461935e-06, ('q3','q3'):3.376657750654264e-06, ('q4','q4'):1.3917138313769865e-06}

quadratic = {('q1','q3'):1.6529989434797471e-06, ('q1','q4'):-1.6529989434797471e-06, ('q2','q3'):-1.6529989434797471e-06, ('q2','q4'):1.6529989434797471e-06} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  0  0  0  0      0.0     998     0.0
1  1  0  0  1 0.000001       2     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [148]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -12
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699072265625000000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69848632812500000000E+01
Previous error of x is 1.54619698150781914592E-04
Previous error of y is -1.55229439442905459146E-04


# Matrix Q is
[[ 1.51499574e-07  0.00000000e+00  4.13249736e-07 -4.13249736e-07]
 [ 0.00000000e+00  6.82965453e-07 -4.13249736e-07  4.13249736e-07]
 [ 0.00000000e+00  0.00000000e+00  1.09228243e-06  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.98104679e-08]]

Minimum energy is  -2.4190562296445567e-07



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.5149957355223582e-07, ('q2','q2'):6.829654533032331e-07, ('q3','q3'):1.0922824275732255e-06, ('q4','q4'):9.981046793458696e-08}

quadratic = {('q1','q3'):4.132497358699368e-07, ('q1','q4'):-4.13249735

In [149]:
linear = {('q1','q1'):1.5149957355223582e-07, ('q2','q2'):6.829654533032331e-07, ('q3','q3'):1.0922824275732255e-06, ('q4','q4'):9.981046793458696e-08}

quadratic = {('q1','q3'):4.132497358699368e-07, ('q1','q4'):-4.132497358699368e-07, ('q2','q3'):-4.132497358699368e-07, ('q2','q4'):4.132497358699368e-07} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  1  0  0  1   -0.0     992     0.0
1  0  0  0  0    0.0       7     0.0
2  0  0  0  1    0.0       1     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [150]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -12
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699096679687500000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69851074218750000000E+01
Previous error of x is -8.95209268492180854082E-05
Previous error of y is 8.89111855570945408545E-05


# Matrix Q is
[[ 5.72714865e-07  0.00000000e+00  4.13249736e-07 -4.13249736e-07]
 [ 0.00000000e+00  2.61750162e-07 -4.13249736e-07  4.13249736e-07]
 [ 0.00000000e+00  0.00000000e+00  3.13439268e-07  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  8.78653627e-07]]

Minimum energy is  -7.996592863261828e-08



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):5.727148650948904e-07, ('q2','q2'):2.617501617605785e-07, ('q3','q3'):3.1343926835549276e-07, ('q4','q4'):8.786536271523199e-07}

quadratic = {('q1','q3'):4.132497358699368e-07, ('q1','q4'):-4.1324973586

In [151]:
linear = {('q1','q1'):5.727148650948904e-07, ('q2','q2'):2.617501617605785e-07, ('q3','q3'):3.1343926835549276e-07, ('q4','q4'):8.786536271523199e-07}

quadratic = {('q1','q3'):4.132497358699368e-07, ('q1','q4'):-4.132497358699368e-07, ('q2','q3'):-4.132497358699368e-07, ('q2','q4'):4.132497358699368e-07} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  0  0  0  0      0.0     974     0.0
1  0  1  1  0      0.0      23     0.0
2  1  0  0  0 0.000001       1     0.0
3  0  0  0  1 0.000001       2     0.0
['BINARY', 4 rows, 1000 samples, 4 variables]


In [152]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -13
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699096679687500000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69851074218750000000E+01
Previous error of x is -8.95209268492180854082E-05
Previous error of y is 8.89111855570945408545E-05


# Matrix Q is
[[ 1.82049304e-07  0.00000000e+00  1.03312434e-07 -1.03312434e-07]
 [ 0.00000000e+00  2.65669525e-08 -1.03312434e-07  1.03312434e-07]
 [ 0.00000000e+00  0.00000000e+00  7.70802224e-09  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.90315202e-07]]

Minimum energy is  -7.996592863261828e-08



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.820493041905116e-07, ('q2','q2'):2.656695252335562e-08, ('q3','q3'):7.708022239269796e-09, ('q4','q4'):2.903152016376834e-07}

quadratic = {('q1','q3'):1.033124339674842e-07, ('q1','q4'):-1.03312433967

In [153]:
linear = {('q1','q1'):1.820493041905116e-07, ('q2','q2'):2.656695252335562e-08, ('q3','q3'):7.708022239269796e-09, ('q4','q4'):2.903152016376834e-07}

quadratic = {('q1','q3'):1.033124339674842e-07, ('q1','q4'):-1.033124339674842e-07, ('q2','q3'):-1.033124339674842e-07, ('q2','q4'):1.033124339674842e-07} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  1  1  0   -0.0     996     0.0
1  0  0  0  0    0.0       2     0.0
2  0  1  0  0    0.0       2     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [154]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -13
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699084472656250000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69849853515625000000E+01
Previous error of x is 3.25493856507819145918E-05
Previous error of y is -3.31591269429054591455E-05


# Matrix Q is
[[ 7.67454815e-08  0.00000000e+00  1.03312434e-07 -1.03312434e-07]
 [ 0.00000000e+00  1.31870775e-07 -1.03312434e-07  1.03312434e-07]
 [ 0.00000000e+00  0.00000000e+00  2.02418812e-07  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.56044120e-08]]

Minimum energy is  -1.092846934226512e-08



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):7.67454814618572e-08, ('q2','q2'):1.3187077525201002e-07, ('q3','q3'):2.0241881185140674e-07, ('q4','q4'):9.560441202554641e-08}

quadratic = {('q1','q3'):1.033124339674842e-07, ('q1','q4'):-1.0331243396

In [155]:
linear = {('q1','q1'):7.67454814618572e-08, ('q2','q2'):1.3187077525201002e-07, ('q3','q3'):2.0241881185140674e-07, ('q4','q4'):9.560441202554641e-08}

quadratic = {('q1','q3'):1.033124339674842e-07, ('q1','q4'):-1.033124339674842e-07, ('q2','q3'):-1.033124339674842e-07, ('q2','q4'):1.033124339674842e-07} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [156]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -14
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699084472656250000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69849853515625000000E+01
Previous error of x is 3.25493856507819145918E-05
Previous error of y is -3.31591269429054591455E-05


# Matrix Q is
[[ 1.22957086e-08  0.00000000e+00  2.58281085e-08 -2.58281085e-08]
 [ 0.00000000e+00  3.98583555e-08 -2.58281085e-08  2.58281085e-08]
 [ 0.00000000e+00  0.00000000e+00  6.39565029e-08  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.05493030e-08]]

Minimum energy is  -1.092846934226512e-08



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.2295708641695191e-08, ('q2','q2'):3.985835553677161e-08, ('q3','q3'):6.395650294108423e-08, ('q4','q4'):1.0549303028154053e-08}

quadratic = {('q1','q3'):2.582810849187105e-08, ('q1','q4'):-2.582810849

In [157]:
linear = {('q1','q1'):1.2295708641695191e-08, ('q2','q2'):3.985835553677161e-08, ('q3','q3'):6.395650294108423e-08, ('q4','q4'):1.0549303028154053e-08}

quadratic = {('q1','q3'):2.582810849187105e-08, ('q1','q4'):-2.582810849187105e-08, ('q2','q3'):-2.582810849187105e-08, ('q2','q4'):2.582810849187105e-08} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  1  0  0  1   -0.0     868     0.0
1  0  0  0  0    0.0     131     0.0
2  0  0  0  1    0.0       1     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [158]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -14
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699090576171875000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850463867187500000E+01
Previous error of x is -2.84857705992180854082E-05
Previous error of y is 2.78760293070945408545E-05


# Matrix Q is
[[ 3.86216647e-08  0.00000000e+00  2.58281085e-08 -2.58281085e-08]
 [ 0.00000000e+00  1.35323995e-08 -2.58281085e-08  2.58281085e-08]
 [ 0.00000000e+00  0.00000000e+00  1.52788057e-08  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.92270002e-08]]

Minimum energy is  -7.94537254648374e-09



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):3.862166465061683e-08, ('q2','q2'):1.3532399527849972e-08, ('q3','q3'):1.5278805735639255e-08, ('q4','q4'):5.922700023359903e-08}

quadratic = {('q1','q3'):2.582810849187105e-08, ('q1','q4'):-2.5828108491

In [159]:
linear = {('q1','q1'):3.862166465061683e-08, ('q2','q2'):1.3532399527849972e-08, ('q3','q3'):1.5278805735639255e-08, ('q4','q4'):5.922700023359903e-08}

quadratic = {('q1','q3'):2.582810849187105e-08, ('q1','q4'):-2.582810849187105e-08, ('q2','q3'):-2.582810849187105e-08, ('q2','q4'):2.582810849187105e-08} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     705     0.0
1  0  1  1  0    0.0     294     0.0
2  0  1  0  0    0.0       1     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [160]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -15
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699090576171875000E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850463867187500000E+01
Previous error of x is -2.84857705992180854082E-05
Previous error of y is 2.78760293070945408545E-05


# Matrix Q is
[[ 1.27915743e-08  0.00000000e+00  6.45702712e-09 -6.45702712e-09]
 [ 0.00000000e+00  2.46941742e-10 -6.45702712e-09  6.45702712e-09]
 [ 0.00000000e+00  0.00000000e+00 -1.67382288e-09  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.03002744e-08]]

Minimum energy is  -7.94537254648374e-09



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.2791574303000066e-08, ('q2','q2'):2.4694174161663483e-10, ('q3','q3'):-1.6738228783351565e-09, ('q4','q4'):2.0300274370644728e-08}

quadratic = {('q1','q3'):6.457027122967762e-09, ('q1','q4'):-6.4570271

In [161]:
linear = {('q1','q1'):1.2791574303000066e-08, ('q2','q2'):2.4694174161663483e-10, ('q3','q3'):-1.6738228783351565e-09, ('q4','q4'):2.0300274370644728e-08}

quadratic = {('q1','q3'):6.457027122967762e-09, ('q1','q4'):-6.457027122967762e-09, ('q2','q3'):-6.457027122967762e-09, ('q2','q4'):6.457027122967762e-09} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  1  1  0   -0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [162]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -15
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087524414062500E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850158691406250000E+01
Previous error of x is 2.03180752578191459179E-06
Previous error of y is -2.64154881790545914555E-06


# Matrix Q is
[[ 6.21008530e-09  0.00000000e+00  6.45702712e-09 -6.45702712e-09]
 [ 0.00000000e+00  6.82843074e-09 -6.45702712e-09  6.45702712e-09]
 [ 0.00000000e+00  0.00000000e+00  1.04956014e-08  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  8.13085007e-09]]

Minimum energy is  -6.146428627815108e-11



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):6.210085300769655e-09, ('q2','q2'):6.828430743847047e-09, ('q3','q3'):1.0495601423026085e-08, ('q4','q4'):8.130850069283485e-09}

quadratic = {('q1','q3'):6.457027122967762e-09, ('q1','q4'):-6.4570271229

In [163]:
linear = {('q1','q1'):6.210085300769655e-09, ('q2','q2'):6.828430743847047e-09, ('q3','q3'):1.0495601423026085e-08, ('q4','q4'):8.130850069283485e-09}

quadratic = {('q1','q3'):6.457027122967762e-09, ('q1','q4'):-6.457027122967762e-09, ('q2','q3'):-6.457027122967762e-09, ('q2','q4'):6.457027122967762e-09} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     999     0.0
1  0  1  0  0    0.0       1     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [164]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -16
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087524414062500E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850158691406250000E+01
Previous error of x is 2.03180752578191459179E-06
Previous error of y is -2.64154881790545914555E-06


# Matrix Q is
[[ 1.47522814e-09  0.00000000e+00  1.61425678e-09 -1.61425678e-09]
 [ 0.00000000e+00  1.78440087e-09 -1.61425678e-09  1.61425678e-09]
 [ 0.00000000e+00  0.00000000e+00  2.91949427e-09  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.73711860e-09]]

Minimum energy is  -6.146428627815108e-11



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.47522814480774e-09, ('q2','q2'):1.7844008663464354e-09, ('q3','q3'):2.9194942749743466e-09, ('q4','q4'):1.7371185981030462e-09}

quadratic = {('q1','q3'):1.6142567807419406e-09, ('q1','q4'):-1.61425678

In [165]:
linear = {('q1','q1'):1.47522814480774e-09, ('q2','q2'):1.7844008663464354e-09, ('q3','q3'):2.9194942749743466e-09, ('q4','q4'):1.7371185981030462e-09}

quadratic = {('q1','q3'):1.6142567807419406e-09, ('q1','q4'):-1.6142567807419406e-09, ('q2','q3'):-1.6142567807419406e-09, ('q2','q4'):1.6142567807419406e-09} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [166]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -17
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087524414062500E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850158691406250000E+01
Previous error of x is 2.03180752578191459179E-06
Previous error of y is -2.64154881790545914555E-06


# Matrix Q is
[[ 3.30160446e-10  0.00000000e+00  4.03564195e-10 -4.03564195e-10]
 [ 0.00000000e+00  4.84746807e-10 -4.03564195e-10  4.03564195e-10]
 [ 0.00000000e+00  0.00000000e+00  8.77670528e-10  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.86482690e-10]]

Minimum energy is  -6.146428627815108e-11



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):3.3016044600959807e-10, ('q2','q2'):4.847468067789458e-10, ('q3','q3'):8.776705283524991e-10, ('q4','q4'):2.86482689916849e-10}

quadratic = {('q1','q3'):4.0356419518548514e-10, ('q1','q4'):-4.0356419518

In [167]:
linear = {('q1','q1'):3.3016044600959807e-10, ('q2','q2'):4.847468067789458e-10, ('q3','q3'):8.776705283524991e-10, ('q4','q4'):2.86482689916849e-10}

quadratic = {('q1','q3'):4.0356419518548514e-10, ('q1','q4'):-4.0356419518548514e-10, ('q2','q3'):-4.0356419518548514e-10, ('q2','q4'):4.0356419518548514e-10} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     991     0.0
1  1  0  0  1    0.0       8     0.0
2  1  0  0  0    0.0       1     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [168]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -18
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087524414062500E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850158691406250000E+01
Previous error of x is 2.03180752578191459179E-06
Previous error of y is -2.64154881790545914555E-06


# Matrix Q is
[[ 6.32168164e-11  0.00000000e+00  1.00891049e-10 -1.00891049e-10]
 [ 0.00000000e+00  1.40509997e-10 -1.00891049e-10  1.00891049e-10]
 [ 0.00000000e+00  0.00000000e+00  2.93316112e-10  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.27780733e-12]]

Minimum energy is  -6.146428627815108e-11



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):6.321681640623105e-11, ('q2','q2'):1.405099967909049e-10, ('q3','q3'):2.9331611189258104e-10, ('q4','q4'):-2.277807325244016e-12}

quadratic = {('q1','q3'):1.0089104879637129e-10, ('q1','q4'):-1.00891048

In [169]:
linear = {('q1','q1'):6.321681640623105e-11, ('q2','q2'):1.405099967909049e-10, ('q3','q3'):2.9331611189258104e-10, ('q4','q4'):-2.277807325244016e-12}

quadratic = {('q1','q3'):1.0089104879637129e-10, ('q1','q4'):-1.0089104879637129e-10, ('q2','q3'):-1.0089104879637129e-10, ('q2','q4'):1.0089104879637129e-10} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  1  0  0  1   -0.0     972     0.0
1  0  0  0  1   -0.0      15     0.0
2  0  0  0  0    0.0      13     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [170]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -18
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087905883789062E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850196838378906250E+01
Previous error of x is -1.78288973984308540821E-06
Previous error of y is 1.17314844771954085445E-06


# Matrix Q is
[[ 1.66052576e-10  0.00000000e+00  1.00891049e-10 -1.00891049e-10]
 [ 0.00000000e+00  3.76742376e-11 -1.00891049e-10  1.00891049e-10]
 [ 0.00000000e+00  0.00000000e+00  1.03168879e-10  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.87869425e-10]]

Minimum energy is  -2.1512237073652864e-11



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.6605257561817332e-10, ('q2','q2'):3.767423757896265e-11, ('q3','q3'):1.0316887909142801e-10, ('q4','q4'):1.8786942547590905e-10}

quadratic = {('q1','q3'):1.0089104879637129e-10, ('q1','q4'):-1.008910

In [171]:
linear = {('q1','q1'):1.6605257561817332e-10, ('q2','q2'):3.767423757896265e-11, ('q3','q3'):1.0316887909142801e-10, ('q4','q4'):1.8786942547590905e-10}

quadratic = {('q1','q3'):1.0089104879637129e-10, ('q1','q4'):-1.0089104879637129e-10, ('q2','q3'):-1.0089104879637129e-10, ('q2','q4'):1.0089104879637129e-10} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     995     0.0
1  0  1  0  0    0.0       5     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [172]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -19
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087905883789062E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850196838378906250E+01
Previous error of x is -1.78288973984308540821E-06
Previous error of y is 1.17314844771954085445E-06


# Matrix Q is
[[ 5.75604362e-11  0.00000000e+00  2.52227622e-11 -2.52227622e-11]
 [ 0.00000000e+00 -6.62873286e-12 -2.52227622e-11  2.52227622e-11]
 [ 0.00000000e+00  0.00000000e+00  1.52046515e-11  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.75549247e-11]]

Minimum energy is  -2.1512237073652864e-11



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):5.7560436159444665e-11, ('q2','q2'):-6.6287328601606724e-12, ('q3','q3'):1.520465147479687e-11, ('q4','q4'):5.7554924667037396e-11}

quadratic = {('q1','q3'):2.522276219909282e-11, ('q1','q4'):-2.522276

In [173]:
linear = {('q1','q1'):5.7560436159444665e-11, ('q2','q2'):-6.6287328601606724e-12, ('q3','q3'):1.520465147479687e-11, ('q4','q4'):5.7554924667037396e-11}

quadratic = {('q1','q3'):2.522276219909282e-11, ('q1','q4'):-2.522276219909282e-11, ('q2','q3'):-2.522276219909282e-11, ('q2','q4'):2.522276219909282e-11} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  1  1  0   -0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [174]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -19
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087715148925781E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850177764892578125E+01
Previous error of x is 1.24458892969414591789E-07
Previous error of y is -7.34200185092959145550E-07


# Matrix Q is
[[ 3.18514886e-11  0.00000000e+00  2.52227622e-11 -2.52227622e-11]
 [ 0.00000000e+00  1.90802147e-11 -2.52227622e-11  2.52227622e-11]
 [ 0.00000000e+00  0.00000000e+00  6.27414505e-11  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00181256e-11]]

Minimum energy is  -4.865388166910008e-12



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):3.185148859853987e-11, ('q2','q2'):1.9080214700744128e-11, ('q3','q3'):6.274145049579132e-11, ('q4','q4'):1.0018125646042946e-11}

quadratic = {('q1','q3'):2.522276219909282e-11, ('q1','q4'):-2.522276219

In [175]:
linear = {('q1','q1'):3.185148859853987e-11, ('q2','q2'):1.9080214700744128e-11, ('q3','q3'):6.274145049579132e-11, ('q4','q4'):1.0018125646042946e-11}

quadratic = {('q1','q3'):2.522276219909282e-11, ('q1','q4'):-2.522276219909282e-11, ('q2','q3'):-2.522276219909282e-11, ('q2','q4'):2.522276219909282e-11} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     995     0.0
1  0  0  0  1    0.0       5     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [176]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -20
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087715148925781E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850177764892578125E+01
Previous error of x is 1.24458892969414591789E-07
Previous error of y is -7.34200185092959145550E-07


# Matrix Q is
[[ 9.55928139e-12  0.00000000e+00  6.30569055e-12 -6.30569055e-12]
 [ 0.00000000e+00  3.17364444e-12 -6.30569055e-12  6.30569055e-12]
 [ 0.00000000e+00  0.00000000e+00  2.22757782e-11  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.08588419e-12]]

Minimum energy is  -4.865388166910008e-12



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):9.559281386859435e-12, ('q2','q2'):3.173644437961563e-12, ('q3','q3'):2.2275778230166377e-11, ('q4','q4'):-4.0858841947078095e-12}

quadratic = {('q1','q3'):6.305690549773205e-12, ('q1','q4'):-6.30569054

In [177]:
linear = {('q1','q1'):9.559281386859435e-12, ('q2','q2'):3.173644437961563e-12, ('q3','q3'):2.2275778230166377e-11, ('q4','q4'):-4.0858841947078095e-12}

quadratic = {('q1','q3'):6.305690549773205e-12, ('q1','q4'):-6.305690549773205e-12, ('q2','q3'):-6.305690549773205e-12, ('q2','q4'):6.305690549773205e-12} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  1   -0.0     987     0.0
1  1  0  0  1   -0.0      12     0.0
2  0  0  0  0    0.0       1     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [178]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -20
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087715148925781E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850187301635742188E+01
Previous error of x is 1.24458892969414591789E-07
Previous error of y is 2.19474131313290854450E-07


# Matrix Q is
[[ 3.25359233e-12  0.00000000e+00  6.30569055e-12 -6.30569055e-12]
 [ 0.00000000e+00  9.47933349e-12 -6.30569055e-12  6.30569055e-12]
 [ 0.00000000e+00  0.00000000e+00  4.08588542e-12  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.41040086e-11]]

Minimum energy is  -7.795034959757894e-13



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):3.2535923312282542e-12, ('q2','q2'):9.479333493592744e-12, ('q3','q3'):4.085885416743183e-12, ('q4','q4'):1.4104008618715382e-11}

quadratic = {('q1','q3'):6.305690549773205e-12, ('q1','q4'):-6.3056905497

In [179]:
linear = {('q1','q1'):3.2535923312282542e-12, ('q2','q2'):9.479333493592744e-12, ('q3','q3'):4.085885416743183e-12, ('q4','q4'):1.4104008618715382e-11}

quadratic = {('q1','q3'):6.305690549773205e-12, ('q1','q4'):-6.305690549773205e-12, ('q2','q3'):-6.305690549773205e-12, ('q2','q4'):6.305690549773205e-12} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [180]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -21
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087715148925781E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850187301635742188E+01
Previous error of x is 1.24458892969414591789E-07
Previous error of y is 2.19474131313290854450E-07


# Matrix Q is
[[ 3.51804375e-14  0.00000000e+00  1.57642264e-12 -1.57642264e-12]
 [ 0.00000000e+00  3.14805102e-12 -1.57642264e-12  1.57642264e-12]
 [ 0.00000000e+00  0.00000000e+00 -2.30794046e-13  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.77826755e-12]]

Minimum energy is  -7.795034959757894e-13



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):3.5180437511502685e-14, ('q2','q2'):3.148051018693747e-12, ('q3','q3'):-2.307940460607292e-13, ('q4','q4'):4.7782675549253704e-12}

quadratic = {('q1','q3'):1.5764226374433013e-12, ('q1','q4'):-1.57642263

In [181]:
linear = {('q1','q1'):3.5180437511502685e-14, ('q2','q2'):3.148051018693747e-12, ('q3','q3'):-2.307940460607292e-13, ('q4','q4'):4.7782675549253704e-12}

quadratic = {('q1','q3'):1.5764226374433013e-12, ('q1','q4'):-1.5764226374433013e-12, ('q2','q3'):-1.5764226374433013e-12, ('q2','q4'):1.5764226374433013e-12} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  1  0   -0.0     805     0.0
1  0  0  0  0    0.0     125     0.0
2  1  0  0  0    0.0      70     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [182]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -21
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087715148925781E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850182533264160156E+01
Previous error of x is 1.24458892969414591789E-07
Previous error of y is -2.57363026889834145550E-07


# Matrix Q is
[[ 1.61160428e-12  0.00000000e+00  1.57642264e-12 -1.57642264e-12]
 [ 0.00000000e+00  1.57162717e-12 -1.57642264e-12  1.57642264e-12]
 [ 0.00000000e+00  0.00000000e+00  4.31667955e-12  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.30793955e-13]]

Minimum energy is  -5.487091831965807e-13



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.6116042844765682e-12, ('q2','q2'):1.5716271717286816e-12, ('q3','q3'):4.316679553549497e-12, ('q4','q4'):2.3079395531514407e-13}

quadratic = {('q1','q3'):1.5764226374433013e-12, ('q1','q4'):-1.5764226

In [183]:
linear = {('q1','q1'):1.6116042844765682e-12, ('q2','q2'):1.5716271717286816e-12, ('q3','q3'):4.316679553549497e-12, ('q4','q4'):2.3079395531514407e-13}

quadratic = {('q1','q3'):1.5764226374433013e-12, ('q1','q4'):-1.5764226374433013e-12, ('q2','q3'):-1.5764226374433013e-12, ('q2','q4'):1.5764226374433013e-12} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     722     0.0
1  0  0  0  1    0.0     140     0.0
2  1  0  0  1    0.0     138     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [184]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -22
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087715148925781E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850182533264160156E+01
Previous error of x is 1.24458892969414591789E-07
Previous error of y is -2.57363026889834145550E-07


# Matrix Q is
[[ 4.07898210e-13  0.00000000e+00  3.94105659e-13 -3.94105659e-13]
 [ 0.00000000e+00  3.87909654e-13 -3.94105659e-13  3.94105659e-13]
 [ 0.00000000e+00  0.00000000e+00  1.58990559e-12  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.53037211e-13]]

Minimum energy is  -5.487091831965807e-13



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):4.0789821021262784e-13, ('q2','q2'):3.879096538386845e-13, ('q3','q3'):1.5899055881666683e-12, ('q4','q4'):-4.5303721095050806e-13}

quadratic = {('q1','q3'):3.9410565936082534e-13, ('q1','q4'):-3.941056

In [185]:
linear = {('q1','q1'):4.0789821021262784e-13, ('q2','q2'):3.879096538386845e-13, ('q3','q3'):1.5899055881666683e-12, ('q4','q4'):-4.5303721095050806e-13}

quadratic = {('q1','q3'):3.9410565936082534e-13, ('q1','q4'):-3.9410565936082534e-13, ('q2','q3'):-3.9410565936082534e-13, ('q2','q4'):3.9410565936082534e-13} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  1   -0.0     476     0.0
1  1  0  0  1   -0.0     524     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [186]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -22
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087715148925781E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850184917449951172E+01
Previous error of x is 1.24458892969414591789E-07
Previous error of y is -1.89444477882716455497E-08


# Matrix Q is
[[ 1.37925551e-14  0.00000000e+00  3.94105659e-13 -3.94105659e-13]
 [ 0.00000000e+00  7.82015309e-13 -3.94105659e-13  3.94105659e-13]
 [ 0.00000000e+00  0.00000000e+00  4.53036813e-13  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.83831565e-13]]

Minimum energy is  -9.567193836745845e-14



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.3792555130044804e-14, ('q2','q2'):7.820153089212676e-13, ('q3','q3'):4.5303681268546227e-13, ('q4','q4'):6.838315645306981e-13}

quadratic = {('q1','q3'):3.9410565936082534e-13, ('q1','q4'):-3.94105659

In [187]:
linear = {('q1','q1'):1.3792555130044804e-14, ('q2','q2'):7.820153089212676e-13, ('q3','q3'):4.5303681268546227e-13, ('q4','q4'):6.838315645306981e-13}

quadratic = {('q1','q3'):3.9410565936082534e-13, ('q1','q4'):-3.9410565936082534e-13, ('q2','q3'):-3.9410565936082534e-13, ('q2','q4'):3.9410565936082534e-13} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     679     0.0
1  1  0  0  0    0.0     321     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [188]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -23
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087715148925781E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850184917449951172E+01
Previous error of x is 1.24458892969414591789E-07
Previous error of y is -1.89444477882716455497E-08


# Matrix Q is
[[-9.25797054e-14  0.00000000e+00  9.85264148e-14 -9.85264148e-14]
 [ 0.00000000e+00  2.91531671e-13 -9.85264148e-14  9.85264148e-14]
 [ 0.00000000e+00  0.00000000e+00  8.44098592e-14  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.99807235e-13]]

Minimum energy is  -9.567193836745845e-14



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-9.257970544139165e-14, ('q2','q2'):2.9153167145421975e-13, ('q3','q3'):8.44098591907111e-14, ('q4','q4'):1.99807235113329e-13}

quadratic = {('q1','q3'):9.852641484020633e-14, ('q1','q4'):-9.85264148402

In [189]:
linear = {('q1','q1'):-9.257970544139165e-14, ('q2','q2'):2.9153167145421975e-13, ('q3','q3'):8.44098591907111e-14, ('q4','q4'):1.99807235113329e-13}

quadratic = {('q1','q3'):9.852641484020633e-14, ('q1','q4'):-9.852641484020633e-14, ('q2','q3'):-9.852641484020633e-14, ('q2','q4'):9.852641484020633e-14} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  1  0  0  0   -0.0     999     0.0
1  1  1  0  0    0.0       1     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [190]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -23
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727069854736E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850184917449951172E+01
Previous error of x is 5.24960341863334178925E-09
Previous error of y is -1.89444477882716455497E-08


# Matrix Q is
[[ 1.06372063e-13  0.00000000e+00  9.85264148e-14 -9.85264148e-14]
 [ 0.00000000e+00  9.25799032e-14 -9.85264148e-14  9.85264148e-14]
 [ 0.00000000e+00  0.00000000e+00  1.82936631e-13  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.01280464e-13]]

Minimum energy is  -3.0922983274455016e-15



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.0637206277298939e-13, ('q2','q2'):9.25799032398387e-14, ('q3','q3'):1.8293663076529266e-13, ('q4','q4'):1.0128046353874744e-13}

quadratic = {('q1','q3'):9.852641484020633e-14, ('q1','q4'):-9.85264148

In [191]:
linear = {('q1','q1'):1.0637206277298939e-13, ('q2','q2'):9.25799032398387e-14, ('q3','q3'):1.8293663076529266e-13, ('q4','q4'):1.0128046353874744e-13}

quadratic = {('q1','q3'):9.852641484020633e-14, ('q1','q4'):-9.852641484020633e-14, ('q2','q3'):-9.852641484020633e-14, ('q2','q4'):9.852641484020633e-14} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     998     0.0
1  0  0  0  1    0.0       2     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [192]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -24
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727069854736E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850184917449951172E+01
Previous error of x is 5.24960341863334178925E-09
Previous error of y is -1.89444477882716455497E-08


# Matrix Q is
[[ 2.83170356e-14  0.00000000e+00  2.46316037e-14 -2.46316037e-14]
 [ 0.00000000e+00  2.14209559e-14 -2.46316037e-14  2.46316037e-14]
 [ 0.00000000e+00  0.00000000e+00  5.59411786e-14  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.51130950e-14]]

Minimum energy is  -3.0922983274455016e-15



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):2.8317035634891185e-14, ('q2','q2'):2.1420955868315837e-14, ('q3','q3'):5.594117859464131e-14, ('q4','q4'):1.5113094981368708e-14}

quadratic = {('q1','q3'):2.4631603710051583e-14, ('q1','q4'):-2.463160

In [193]:
linear = {('q1','q1'):2.8317035634891185e-14, ('q2','q2'):2.1420955868315837e-14, ('q3','q3'):5.594117859464131e-14, ('q4','q4'):1.5113094981368708e-14}

quadratic = {('q1','q3'):2.4631603710051583e-14, ('q1','q4'):-2.4631603710051583e-14, ('q2','q3'):-2.4631603710051583e-14, ('q2','q4'):2.4631603710051583e-14} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [194]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -25
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727069854736E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850184917449951172E+01
Previous error of x is 5.24960341863334178925E-09
Previous error of y is -1.89444477882716455497E-08


# Matrix Q is
[[ 7.94126888e-15  0.00000000e+00  6.15790093e-15 -6.15790093e-15]
 [ 0.00000000e+00  4.49322900e-15 -6.15790093e-15  6.15790093e-15]
 [ 0.00000000e+00  0.00000000e+00  1.90888051e-14  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.32523671e-15]]

Minimum energy is  -3.0922983274455016e-15



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):7.941268879544714e-15, ('q2','q2'):4.493228996257042e-15, ('q3','q3'):1.90888051003194e-14, ('q4','q4'):-1.3252367063168984e-15}

quadratic = {('q1','q3'):6.157900927512896e-15, ('q1','q4'):-6.157900927

In [195]:
linear = {('q1','q1'):7.941268879544714e-15, ('q2','q2'):4.493228996257042e-15, ('q3','q3'):1.90888051003194e-14, ('q4','q4'):-1.3252367063168984e-15}

quadratic = {('q1','q3'):6.157900927512896e-15, ('q1','q4'):-6.157900927512896e-15, ('q2','q3'):-6.157900927512896e-15, ('q2','q4'):6.157900927512896e-15} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  1   -0.0     936     0.0
1  0  0  0  0    0.0      42     0.0
2  1  0  0  1    0.0      22     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [196]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -25
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727069854736E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185215473175049E+01
Previous error of x is 5.24960341863334178925E-09
Previous error of y is 1.08578745994236669503E-08


# Matrix Q is
[[ 1.78337760e-15  0.00000000e+00  6.15790093e-15 -6.15790093e-15]
 [ 0.00000000e+00  1.06511203e-14 -6.15790093e-15  6.15790093e-15]
 [ 0.00000000e+00  0.00000000e+00  1.32521875e-15  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.64383496e-14]]

Minimum energy is  -1.7670664645751183e-15



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.783377601963208e-15, ('q2','q2'):1.0651120273838548e-14, ('q3','q3'):1.3252187465927529e-15, ('q4','q4'):1.6438349647409753e-14}

quadratic = {('q1','q3'):6.157900927512896e-15, ('q1','q4'):-6.15790092

In [197]:
linear = {('q1','q1'):1.783377601963208e-15, ('q2','q2'):1.0651120273838548e-14, ('q3','q3'):1.3252187465927529e-15, ('q4','q4'):1.6438349647409753e-14}

quadratic = {('q1','q3'):6.157900927512896e-15, ('q1','q4'):-6.157900927512896e-15, ('q2','q3'):-6.157900927512896e-15, ('q2','q4'):6.157900927512896e-15} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     918     0.0
1  0  0  1  0    0.0      50     0.0
2  1  0  0  0    0.0      32     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [198]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -26
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727069854736E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185215473175049E+01
Previous error of x is 5.24960341863334178925E-09
Previous error of y is 1.08578745994236669503E-08


# Matrix Q is
[[-6.62623433e-16  0.00000000e+00  1.53947523e-15 -1.53947523e-15]
 [ 0.00000000e+00  3.77124790e-15 -1.53947523e-15  1.53947523e-15]
 [ 0.00000000e+00  0.00000000e+00 -1.55783668e-15  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.99872877e-15]]

Minimum energy is  -1.7670664645751183e-15



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-6.626234334936152e-16, ('q2','q2'):3.771247902444054e-15, ('q3','q3'):-1.5578366759539365e-15, ('q4','q4'):5.998728774454563e-15}

quadratic = {('q1','q3'):1.539475231878224e-15, ('q1','q4'):-1.53947523

In [199]:
linear = {('q1','q1'):-6.626234334936152e-16, ('q2','q2'):3.771247902444054e-15, ('q3','q3'):-1.5578366759539365e-15, ('q4','q4'):5.998728774454563e-15}

quadratic = {('q1','q3'):1.539475231878224e-15, ('q1','q4'):-1.539475231878224e-15, ('q2','q3'):-1.539475231878224e-15, ('q2','q4'):1.539475231878224e-15} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  1  0   -0.0     993     0.0
1  1  0  1  0   -0.0       3     0.0
2  1  0  0  0   -0.0       4     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [200]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -26
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727069854736E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185066461563110E+01
Previous error of x is 5.24960341863334178925E-09
Previous error of y is -4.04328659442398929968E-09


# Matrix Q is
[[ 8.76849386e-16  0.00000000e+00  1.53947523e-15 -1.53947523e-15]
 [ 0.00000000e+00  2.23177508e-15 -1.53947523e-15  1.53947523e-15]
 [ 0.00000000e+00  0.00000000e+00  2.88305991e-15  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.55783219e-15]]

Minimum energy is  -2.0923185682363195e-16



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):8.768493859017611e-16, ('q2','q2'):2.2317750830486782e-15, ('q3','q3'):2.8830599124777257e-15, ('q4','q4'):1.5578321860229003e-15}

quadratic = {('q1','q3'):1.539475231878224e-15, ('q1','q4'):-1.5394752

In [201]:
linear = {('q1','q1'):8.768493859017611e-16, ('q2','q2'):2.2317750830486782e-15, ('q3','q3'):2.8830599124777257e-15, ('q4','q4'):1.5578321860229003e-15}

quadratic = {('q1','q3'):1.539475231878224e-15, ('q1','q4'):-1.539475231878224e-15, ('q2','q3'):-1.539475231878224e-15, ('q2','q4'):1.539475231878224e-15} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [202]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -27
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727069854736E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185066461563110E+01
Previous error of x is 5.24960341863334178925E-09
Previous error of y is -4.04328659442398929968E-09


# Matrix Q is
[[ 4.98466343e-17  0.00000000e+00  3.84868808e-16 -3.84868808e-16]
 [ 0.00000000e+00  7.27309483e-16 -3.84868808e-16  3.84868808e-16]
 [ 0.00000000e+00  0.00000000e+00  8.86418444e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.23804581e-16]]

Minimum energy is  -2.0923185682363195e-16



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):4.9846634332075706e-17, ('q2','q2'):7.27309482905534e-16, ('q3','q3'):8.864184439262849e-16, ('q4','q4'):2.238045806988718e-16}

quadratic = {('q1','q3'):3.84868807969556e-16, ('q1','q4'):-3.84868807969

In [203]:
linear = {('q1','q1'):4.9846634332075706e-17, ('q2','q2'):7.27309482905534e-16, ('q3','q3'):8.864184439262849e-16, ('q4','q4'):2.238045806988718e-16}

quadratic = {('q1','q3'):3.84868807969556e-16, ('q1','q4'):-3.84868807969556e-16, ('q2','q3'):-3.84868807969556e-16, ('q2','q4'):3.84868807969556e-16} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  1  0  0  1   -0.0     987     0.0
1  0  0  0  0    0.0       8     0.0
2  1  0  0  0    0.0       5     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [204]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -27
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727814912796E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185140967369080E+01
Previous error of x is -2.20097717829048633575E-09
Previous error of y is 3.40729400249983882532E-09


# Matrix Q is
[[ 4.42153135e-16  0.00000000e+00  3.84868808e-16 -3.84868808e-16]
 [ 0.00000000e+00  3.35002983e-16 -3.84868808e-16  3.84868808e-16]
 [ 0.00000000e+00  0.00000000e+00  1.61049908e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.49173117e-16]]

Minimum energy is  -9.802648192076545e-17



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):4.421531346284192e-16, ('q2','q2'):3.3500298260919055e-16, ('q3','q3'):1.6104990800533558e-16, ('q4','q4'):9.49173116619821e-16}

quadratic = {('q1','q3'):3.84868807969556e-16, ('q1','q4'):-3.84868807969

In [205]:
linear = {('q1','q1'):4.421531346284192e-16, ('q2','q2'):3.3500298260919055e-16, ('q3','q3'):1.6104990800533558e-16, ('q4','q4'):9.49173116619821e-16}

quadratic = {('q1','q3'):3.84868807969556e-16, ('q1','q4'):-3.84868807969556e-16, ('q2','q3'):-3.84868807969556e-16, ('q2','q4'):3.84868807969556e-16} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     963     0.0
1  0  1  1  0    0.0      34     0.0
2  0  0  1  0    0.0       2     0.0
3  0  0  0  1    0.0       1     0.0
['BINARY', 4 rows, 1000 samples, 4 variables]


In [206]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -28
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727814912796E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185140967369080E+01
Previous error of x is -2.20097717829048633575E-09
Previous error of y is 3.40729400249983882532E-09


# Matrix Q is
[[ 1.23932053e-16  0.00000000e+00  9.62172020e-17 -9.62172020e-17]
 [ 0.00000000e+00  7.03569766e-17 -9.62172020e-17  9.62172020e-17]
 [ 0.00000000e+00  0.00000000e+00 -5.82529241e-17  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.35808680e-16]]

Minimum energy is  -9.802648192076545e-17



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.2393205265950836e-16, ('q2','q2'):7.035697664989408e-17, ('q3','q3'):-5.825292407547679e-17, ('q4','q4'):3.3580868023176597e-16}

quadratic = {('q1','q3'):9.6217201992389e-17, ('q1','q4'):-9.6217201992

In [207]:
linear = {('q1','q1'):1.2393205265950836e-16, ('q2','q2'):7.035697664989408e-17, ('q3','q3'):-5.825292407547679e-17, ('q4','q4'):3.3580868023176597e-16}

quadratic = {('q1','q3'):9.6217201992389e-17, ('q1','q4'):-9.6217201992389e-17, ('q2','q3'):-9.6217201992389e-17, ('q2','q4'):9.6217201992389e-17} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  1  1  0   -0.0     957     0.0
1  0  0  1  0   -0.0      43     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [208]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -28
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727442383766E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185103714466095E+01
Previous error of x is 1.52431312017142772675E-09
Previous error of y is -3.17996295962075237185E-10


# Matrix Q is
[[ 2.58506360e-17  0.00000000e+00  9.62172020e-17 -9.62172020e-17]
 [ 0.00000000e+00  1.68438393e-16 -9.62172020e-17  9.62172020e-17]
 [ 0.00000000e+00  0.00000000e+00  1.23095078e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.54460678e-16]]

Minimum energy is  -1.391806240465905e-17



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):2.5850636043495363e-17, ('q2','q2'):1.6843839326590707e-16, ('q3','q3'):1.2309507832116184e-16, ('q4','q4'):1.5446067783512732e-16}

quadratic = {('q1','q3'):9.6217201992389e-17, ('q1','q4'):-9.621720199

In [209]:
linear = {('q1','q1'):2.5850636043495363e-17, ('q2','q2'):1.6843839326590707e-16, ('q3','q3'):1.2309507832116184e-16, ('q4','q4'):1.5446067783512732e-16}

quadratic = {('q1','q3'):9.6217201992389e-17, ('q1','q4'):-9.6217201992389e-17, ('q2','q3'):-9.6217201992389e-17, ('q2','q4'):9.6217201992389e-17} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     987     0.0
1  1  0  0  0    0.0      13     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [210]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -29
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727442383766E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185103714466095E+01
Previous error of x is 1.52431312017142772675E-09
Previous error of y is -3.17996295962075237185E-10


# Matrix Q is
[[-1.13608106e-17  0.00000000e+00  2.40543005e-17 -2.40543005e-17]
 [ 0.00000000e+00  5.99330680e-17 -2.40543005e-17  2.40543005e-17]
 [ 0.00000000e+00  0.00000000e+00  2.68530696e-17  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.25358694e-17]]

Minimum energy is  -1.391806240465905e-17



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-1.1360810641927622e-17, ('q2','q2'):5.993306796927823e-17, ('q3','q3'):2.6853069641044767e-17, ('q4','q4'):4.2535869398027517e-17}

quadratic = {('q1','q3'):2.405430049809725e-17, ('q1','q4'):-2.4054300

In [211]:
linear = {('q1','q1'):-1.1360810641927622e-17, ('q2','q2'):5.993306796927823e-17, ('q3','q3'):2.6853069641044767e-17, ('q4','q4'):4.2535869398027517e-17}

quadratic = {('q1','q3'):2.405430049809725e-17, ('q1','q4'):-2.405430049809725e-17, ('q2','q3'):-2.405430049809725e-17, ('q2','q4'):2.405430049809725e-17} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  1  0  0  0   -0.0     998     0.0
1  0  0  0  0    0.0       2     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [212]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -29
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727628648281E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185103714466095E+01
Previous error of x is -3.38332029059529304504E-10
Previous error of y is -3.17996295962075237185E-10


# Matrix Q is
[[ 3.72140007e-17  0.00000000e+00  2.40543005e-17 -2.40543005e-17]
 [ 0.00000000e+00  1.13582566e-17 -2.40543005e-17  2.40543005e-17]
 [ 0.00000000e+00  0.00000000e+00  5.09139616e-17  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.84749774e-17]]

Minimum energy is  -2.558841038721551e-18



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):3.7214000705181516e-17, ('q2','q2'):1.1358256622169087e-17, ('q3','q3'):5.0913961625492256e-17, ('q4','q4'):1.8474977413580028e-17}

quadratic = {('q1','q3'):2.405430049809725e-17, ('q1','q4'):-2.405430

In [213]:
linear = {('q1','q1'):3.7214000705181516e-17, ('q2','q2'):1.1358256622169087e-17, ('q3','q3'):5.0913961625492256e-17, ('q4','q4'):1.8474977413580028e-17}

quadratic = {('q1','q3'):2.405430049809725e-17, ('q1','q4'):-2.405430049809725e-17, ('q2','q3'):-2.405430049809725e-17, ('q2','q4'):2.405430049809725e-17} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     999     0.0
1  0  1  0  0    0.0       1     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [214]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -30
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727628648281E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185103714466095E+01
Previous error of x is -3.38332029059529304504E-10
Previous error of y is -3.17996295962075237185E-10


# Matrix Q is
[[ 1.25354682e-17  0.00000000e+00  6.01357512e-18 -6.01357512e-18]
 [ 0.00000000e+00 -3.92403855e-19 -6.01357512e-18  6.01357512e-18]
 [ 0.00000000e+00  0.00000000e+00  1.67833634e-17  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.63871327e-19]]

Minimum energy is  -2.558841038721551e-18



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.2535468186671933e-17, ('q2','q2'):-3.9240385483428143e-19, ('q3','q3'):1.6783363432862093e-17, ('q4','q4'):5.6387132690597985e-19}

quadratic = {('q1','q3'):6.013575124524312e-18, ('q1','q4'):-6.01357

In [215]:
linear = {('q1','q1'):1.2535468186671933e-17, ('q2','q2'):-3.9240385483428143e-19, ('q3','q3'):1.6783363432862093e-17, ('q4','q4'):5.6387132690597985e-19}

quadratic = {('q1','q3'):6.013575124524312e-18, ('q1','q4'):-6.013575124524312e-18, ('q2','q3'):-6.013575124524312e-18, ('q2','q4'):6.013575124524312e-18} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  1  0  0   -0.0     577     0.0
1  0  0  0  0    0.0     333     0.0
2  0  0  0  1    0.0      90     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [216]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -30
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727535516024E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185103714466095E+01
Previous error of x is 5.92990545555949211121E-10
Previous error of y is -3.17996295962075237185E-10


# Matrix Q is
[[ 3.91765350e-19  0.00000000e+00  6.01357512e-18 -6.01357512e-18]
 [ 0.00000000e+00  1.17512990e-17 -6.01357512e-18  6.01357512e-18]
 [ 0.00000000e+00  0.00000000e+00  1.07681404e-17  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.57909432e-18]]

Minimum energy is  -2.1662809723385852e-18



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):3.917653498946483e-19, ('q2','q2'):1.1751298981943004e-17, ('q3','q3'):1.076814043675022e-17, ('q4','q4'):6.579094323017852e-18}

quadratic = {('q1','q3'):6.013575124524312e-18, ('q1','q4'):-6.013575124

In [217]:
linear = {('q1','q1'):3.917653498946483e-19, ('q2','q2'):1.1751298981943004e-17, ('q3','q3'):1.076814043675022e-17, ('q4','q4'):6.579094323017852e-18}

quadratic = {('q1','q3'):6.013575124524312e-18, ('q1','q4'):-6.013575124524312e-18, ('q2','q3'):-6.013575124524312e-18, ('q2','q4'):6.013575124524312e-18} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     752     0.0
1  1  0  0  0    0.0     190     0.0
2  1  0  0  1    0.0      58     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [218]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -31
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727535516024E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185103714466095E+01
Previous error of x is 5.92990545555949211121E-10
Previous error of y is -3.17996295962075237185E-10


# Matrix Q is
[[-1.32200037e-18  0.00000000e+00  1.50339378e-18 -1.50339378e-18]
 [ 0.00000000e+00  4.35776645e-18 -1.50339378e-18  1.50339378e-18]
 [ 0.00000000e+00  0.00000000e+00  3.21566587e-18  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.12114282e-18]]

Minimum energy is  -2.1662809723385852e-18



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-1.3220003665323824e-18, ('q2','q2'):4.357766449491795e-18, ('q3','q3'):3.2156658734041013e-18, ('q4','q4'):1.121142816537917e-18}

quadratic = {('q1','q3'):1.503393781131078e-18, ('q1','q4'):-1.5033937

In [219]:
linear = {('q1','q1'):-1.3220003665323824e-18, ('q2','q2'):4.357766449491795e-18, ('q3','q3'):3.2156658734041013e-18, ('q4','q4'):1.121142816537917e-18}

quadratic = {('q1','q3'):1.503393781131078e-18, ('q1','q4'):-1.503393781131078e-18, ('q2','q3'):-1.503393781131078e-18, ('q2','q4'):1.503393781131078e-18} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  1  0  0  1   -0.0     974     0.0
1  1  0  0  0   -0.0      26     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [220]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -31
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727582082152E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185108371078968E+01
Previous error of x is 1.27329258248209953308E-10
Previous error of y is 1.47664991345664020628E-10


# Matrix Q is
[[ 2.10048262e-19  0.00000000e+00  1.50339378e-18 -1.50339378e-18]
 [ 0.00000000e+00  2.82571782e-18 -1.50339378e-18  1.50339378e-18]
 [ 0.00000000e+00  0.00000000e+00  3.81543076e-19  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.95526561e-18]]

Minimum energy is  -4.623426772191642e-19



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):2.1004826228051008e-19, ('q2','q2'):2.8257178206789027e-18, ('q3','q3'):3.815430763239443e-19, ('q4','q4'):3.955265613618074e-18}

quadratic = {('q1','q3'):1.503393781131078e-18, ('q1','q4'):-1.5033937811

In [221]:
linear = {('q1','q1'):2.1004826228051008e-19, ('q2','q2'):2.8257178206789027e-18, ('q3','q3'):3.815430763239443e-19, ('q4','q4'):3.955265613618074e-18}

quadratic = {('q1','q3'):1.503393781131078e-18, ('q1','q4'):-1.503393781131078e-18, ('q2','q3'):-1.503393781131078e-18, ('q2','q4'):1.503393781131078e-18} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     902     0.0
1  1  0  0  0    0.0      77     0.0
2  0  0  1  0    0.0      21     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [222]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -32
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727582082152E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185108371078968E+01
Previous error of x is 1.27329258248209953308E-10
Previous error of y is 1.47664991345664020628E-10


# Matrix Q is
[[-2.74446629e-19  0.00000000e+00  3.75848445e-19 -3.75848445e-19]
 [ 0.00000000e+00  1.03338815e-18 -3.75848445e-19  3.75848445e-19]
 [ 0.00000000e+00  0.00000000e+00 -3.51329548e-19  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.43553172e-18]]

Minimum energy is  -4.623426772191642e-19



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-2.7444662922967156e-19, ('q2','q2'):1.0333881499695248e-18, ('q3','q3'):-3.5132954808078006e-19, ('q4','q4'):1.4355317205662847e-18}

quadratic = {('q1','q3'):3.758484452827695e-19, ('q1','q4'):-3.758484

In [223]:
linear = {('q1','q1'):-2.7444662922967156e-19, ('q2','q2'):1.0333881499695248e-18, ('q3','q3'):-3.5132954808078006e-19, ('q4','q4'):1.4355317205662847e-18}

quadratic = {('q1','q3'):3.758484452827695e-19, ('q1','q4'):-3.758484452827695e-19, ('q2','q3'):-3.758484452827695e-19, ('q2','q4'):3.758484452827695e-19} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  1  0   -0.0     796     0.0
1  1  0  0  0   -0.0     148     0.0
2  1  0  1  0   -0.0      56     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [224]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -32
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727582082152E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106042772532E+01
Previous error of x is 1.27329258248209953308E-10
Previous error of y is -8.51656523082056082785E-11


# Matrix Q is
[[ 1.01696675e-19  0.00000000e+00  3.75848445e-19 -3.75848445e-19]
 [ 0.00000000e+00  6.57244845e-19 -3.75848445e-19  3.75848445e-19]
 [ 0.00000000e+00  0.00000000e+00  7.34079624e-19  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.50122548e-19]]

Minimum energy is  -1.1129384269655966e-19



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.0169667530755038e-19, ('q2','q2'):6.572448454323029e-19, ('q3','q3'):7.340796241040117e-19, ('q4','q4'):3.5012254838149285e-19}

quadratic = {('q1','q3'):3.758484452827695e-19, ('q1','q4'):-3.75848445

In [225]:
linear = {('q1','q1'):1.0169667530755038e-19, ('q2','q2'):6.572448454323029e-19, ('q3','q3'):7.340796241040117e-19, ('q4','q4'):3.5012254838149285e-19}

quadratic = {('q1','q3'):3.758484452827695e-19, ('q1','q4'):-3.758484452827695e-19, ('q2','q3'):-3.758484452827695e-19, ('q2','q4'):3.758484452827695e-19} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     892     0.0
1  1  0  0  1    0.0      98     0.0
2  1  0  0  0    0.0      10     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [226]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -33
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727582082152E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106042772532E+01
Previous error of x is 1.27329258248209953308E-10
Previous error of y is -8.51656523082056082785E-11


# Matrix Q is
[[-4.40193524e-20  0.00000000e+00  9.39621113e-20 -9.39621113e-20]
 [ 0.00000000e+00  2.33754733e-19 -9.39621113e-20  9.39621113e-20]
 [ 0.00000000e+00  0.00000000e+00  2.31514540e-19  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.95360026e-20]]

Minimum energy is  -1.1129384269655966e-19



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-4.4019352438706475e-20, ('q2','q2'):2.3375473262366974e-19, ('q3','q3'):2.3151454049131774e-19, ('q4','q4'):3.953600263005837e-20}

quadratic = {('q1','q3'):9.396211132069238e-20, ('q1','q4'):-9.396211

In [227]:
linear = {('q1','q1'):-4.4019352438706475e-20, ('q2','q2'):2.3375473262366974e-19, ('q3','q3'):2.3151454049131774e-19, ('q4','q4'):3.953600263005837e-20}

quadratic = {('q1','q3'):9.396211132069238e-20, ('q1','q4'):-9.396211132069238e-20, ('q2','q3'):-9.396211132069238e-20, ('q2','q4'):9.396211132069238e-20} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  1  0  0  1   -0.0     995     0.0
1  1  0  0  0   -0.0       1     0.0
2  0  0  0  1    0.0       4     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [228]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -33
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727593723685E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185107206925750E+01
Previous error of x is 1.09139364212751388550E-11
Previous error of y is 3.12496695187292061746E-11


# Matrix Q is
[[ 5.15718018e-20  0.00000000e+00  9.39621113e-20 -9.39621113e-20]
 [ 0.00000000e+00  1.38163578e-19 -9.39621113e-20  9.39621113e-20]
 [ 0.00000000e+00  0.00000000e+00  5.40100419e-20  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.17040501e-19]]

Minimum energy is  -1.2976809449731898e-20



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):5.1571801798555477e-20, ('q2','q2'):1.3816357838640782e-19, ('q3','q3'):5.401004185119825e-20, ('q4','q4'):2.1704050127017786e-19}

quadratic = {('q1','q3'):9.396211132069238e-20, ('q1','q4'):-9.39621113

In [229]:
linear = {('q1','q1'):5.1571801798555477e-20, ('q2','q2'):1.3816357838640782e-19, ('q3','q3'):5.401004185119825e-20, ('q4','q4'):2.1704050127017786e-19}

quadratic = {('q1','q3'):9.396211132069238e-20, ('q1','q4'):-9.396211132069238e-20, ('q2','q3'):-9.396211132069238e-20, ('q2','q4'):9.396211132069238e-20} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [230]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -34
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727593723685E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185107206925750E+01
Previous error of x is 1.09139364212751388550E-11
Previous error of y is 3.12496695187292061746E-11


# Matrix Q is
[[ 2.06897838e-21  0.00000000e+00  2.34905278e-20 -2.34905278e-20]
 [ 0.00000000e+00  4.53648667e-20 -2.34905278e-20  2.34905278e-20]
 [ 0.00000000e+00  0.00000000e+00 -6.87629696e-21  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  7.46389327e-20]]

Minimum energy is  -1.2976809449731898e-20



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):2.068978376157323e-21, ('q2','q2'):4.53648666700835e-20, ('q3','q3'):-6.876296964572888e-21, ('q4','q4'):7.463893274491691e-20}

quadratic = {('q1','q3'):2.3490527830173095e-20, ('q1','q4'):-2.3490527830

In [231]:
linear = {('q1','q1'):2.068978376157323e-21, ('q2','q2'):4.53648666700835e-20, ('q3','q3'):-6.876296964572888e-21, ('q4','q4'):7.463893274491691e-20}

quadratic = {('q1','q3'):2.3490527830173095e-20, ('q1','q4'):-2.3490527830173095e-20, ('q2','q3'):-2.3490527830173095e-20, ('q2','q4'):2.3490527830173095e-20} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  1  0   -0.0     943     0.0
1  0  0  0  0    0.0      44     0.0
2  1  0  0  0    0.0      12     0.0
3  0  1  1  0    0.0       1     0.0
['BINARY', 4 rows, 1000 samples, 4 variables]


In [232]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -34
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727593723685E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106624849141E+01
Previous error of x is 1.09139364212751388550E-11
Previous error of y is -2.69579913947382010520E-11


# Matrix Q is
[[ 2.54595585e-20  0.00000000e+00  2.34905278e-20 -2.34905278e-20]
 [ 0.00000000e+00  2.19742866e-20 -2.34905278e-20  2.34905278e-20]
 [ 0.00000000e+00  0.00000000e+00  6.08217114e-20  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.94092439e-21]]

Minimum energy is  -6.08970366961539e-21



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):2.5459558456656032e-20, ('q2','q2'):2.1974286589584796e-20, ('q3','q3'):6.082171138880704e-20, ('q4','q4'):6.940924391536995e-21}

quadratic = {('q1','q3'):2.3490527830173095e-20, ('q1','q4'):-2.349052783

In [233]:
linear = {('q1','q1'):2.5459558456656032e-20, ('q2','q2'):2.1974286589584796e-20, ('q3','q3'):6.082171138880704e-20, ('q4','q4'):6.940924391536995e-21}

quadratic = {('q1','q3'):2.3490527830173095e-20, ('q1','q4'):-2.3490527830173095e-20, ('q2','q3'):-2.3490527830173095e-20, ('q2','q4'):2.3490527830173095e-20} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     967     0.0
1  0  0  0  1    0.0      26     0.0
2  1  0  0  1    0.0       7     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [234]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -35
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727593723685E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106624849141E+01
Previous error of x is 1.09139364212751388550E-11
Previous error of y is -2.69579913947382010520E-11


# Matrix Q is
[[ 6.80054860e-21  0.00000000e+00  5.87263196e-21 -5.87263196e-21]
 [ 0.00000000e+00  5.05791266e-21 -5.87263196e-21  5.87263196e-21]
 [ 0.00000000e+00  0.00000000e+00  2.19405262e-20  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.99986728e-21]]

Minimum energy is  -6.08970366961539e-21



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):6.800548597547912e-21, ('q2','q2'):5.0579126640122954e-21, ('q3','q3'):2.1940526221860514e-20, ('q4','q4'):-4.999867276774507e-21}

quadratic = {('q1','q3'):5.872631957543274e-21, ('q1','q4'):-5.872631957

In [235]:
linear = {('q1','q1'):6.800548597547912e-21, ('q2','q2'):5.0579126640122954e-21, ('q3','q3'):2.1940526221860514e-20, ('q4','q4'):-4.999867276774507e-21}

quadratic = {('q1','q3'):5.872631957543274e-21, ('q1','q4'):-5.872631957543274e-21, ('q2','q3'):-5.872631957543274e-21, ('q2','q4'):5.872631957543274e-21} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  1   -0.0     752     0.0
1  1  0  0  1   -0.0     248     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [236]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -35
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727593723685E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106915887445E+01
Previous error of x is 1.09139364212751388550E-11
Previous error of y is 2.14583906199550256133E-12


# Matrix Q is
[[ 1.04952573e-21  0.00000000e+00  5.87263196e-21 -5.87263196e-21]
 [ 0.00000000e+00  1.08089355e-20 -5.87263196e-21  5.87263196e-21]
 [ 0.00000000e+00  0.00000000e+00  5.04020958e-21  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.19004494e-20]]

Minimum energy is  -1.0405932105917088e-21



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.0495257252678143e-21, ('q2','q2'):1.0808935536292393e-20, ('q3','q3'):5.040209584465027e-21, ('q4','q4'):1.1900449360620982e-20}

quadratic = {('q1','q3'):5.872631957543274e-21, ('q1','q4'):-5.87263195

In [237]:
linear = {('q1','q1'):1.0495257252678143e-21, ('q2','q2'):1.0808935536292393e-20, ('q3','q3'):5.040209584465027e-21, ('q4','q4'):1.1900449360620982e-20}

quadratic = {('q1','q3'):5.872631957543274e-21, ('q1','q4'):-5.872631957543274e-21, ('q2','q3'):-5.872631957543274e-21, ('q2','q4'):5.872631957543274e-21} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     970     0.0
1  1  0  0  0    0.0      30     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [238]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -36
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727593723685E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106915887445E+01
Previous error of x is 1.09139364212751388550E-11
Previous error of y is 2.14583906199550256133E-12


# Matrix Q is
[[-9.57544795e-22  0.00000000e+00  1.46815799e-21 -1.46815799e-21]
 [ 0.00000000e+00  3.92216011e-21 -1.46815799e-21  1.46815799e-21]
 [ 0.00000000e+00  0.00000000e+00  4.02522424e-22  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.83264231e-21]]

Minimum energy is  -1.0405932105917088e-21



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-9.575447950611188e-22, ('q2','q2'):3.922160110451171e-21, ('q3','q3'):4.025224240967617e-22, ('q4','q4'):3.83264231217474e-21}

quadratic = {('q1','q3'):1.4681579893858185e-21, ('q1','q4'):-1.4681579893

In [239]:
linear = {('q1','q1'):-9.575447950611188e-22, ('q2','q2'):3.922160110451171e-21, ('q3','q3'):4.025224240967617e-22, ('q4','q4'):3.83264231217474e-21}

quadratic = {('q1','q3'):1.4681579893858185e-21, ('q1','q4'):-1.4681579893858185e-21, ('q2','q3'):-1.4681579893858185e-21, ('q2','q4'):1.4681579893858185e-21} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  1  0  0  0   -0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [240]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -36
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727595178876E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106915887445E+01
Previous error of x is -3.63797880709171295166E-12
Previous error of y is 2.14583906199550256133E-12


# Matrix Q is
[[ 2.03421155e-21  0.00000000e+00  1.46815799e-21 -1.46815799e-21]
 [ 0.00000000e+00  9.30403765e-22 -1.46815799e-21  1.46815799e-21]
 [ 0.00000000e+00  0.00000000e+00  1.86920970e-21  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.36595504e-21]]

Minimum energy is  -9.016268676828002e-23



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):2.034211550368985e-21, ('q2','q2'):9.304037650210668e-22, ('q3','q3'):1.869209699037199e-21, ('q4','q4'):2.3659550372343026e-21}

quadratic = {('q1','q3'):1.4681579893858185e-21, ('q1','q4'):-1.468157989

In [241]:
linear = {('q1','q1'):2.034211550368985e-21, ('q2','q2'):9.304037650210668e-22, ('q3','q3'):1.869209699037199e-21, ('q4','q4'):2.3659550372343026e-21}

quadratic = {('q1','q3'):1.4681579893858185e-21, ('q1','q4'):-1.4681579893858185e-21, ('q2','q3'):-1.4681579893858185e-21, ('q2','q4'):1.4681579893858185e-21} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [242]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -37
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727595178876E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106915887445E+01
Previous error of x is -3.63797880709171295166E-12
Previous error of y is 2.14583906199550256133E-12


# Matrix Q is
[[ 6.46528861e-22  0.00000000e+00  3.67039497e-22 -3.67039497e-22]
 [ 0.00000000e+00  9.46249681e-23 -3.67039497e-22  3.67039497e-22]
 [ 0.00000000e+00  0.00000000e+00  4.05209257e-22  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.53581927e-22]]

Minimum energy is  -9.016268676828002e-23



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):6.46528860760736e-22, ('q2','q2'):9.462496808677695e-23, ('q3','q3'):4.0520925748466176e-22, ('q4','q4'):6.535819265832138e-22}

quadratic = {('q1','q3'):3.670394973464546e-22, ('q1','q4'):-3.67039497346

In [243]:
linear = {('q1','q1'):6.46528860760736e-22, ('q2','q2'):9.462496808677695e-23, ('q3','q3'):4.0520925748466176e-22, ('q4','q4'):6.535819265832138e-22}

quadratic = {('q1','q3'):3.670394973464546e-22, ('q1','q4'):-3.670394973464546e-22, ('q2','q3'):-3.670394973464546e-22, ('q2','q4'):3.670394973464546e-22} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     993     0.0
1  0  1  0  0    0.0       6     0.0
2  0  1  1  0    0.0       1     0.0
['BINARY', 3 rows, 1000 samples, 4 variables]


In [244]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -38
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727595178876E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106915887445E+01
Previous error of x is -3.63797880709171295166E-12
Previous error of y is 2.14583906199550256133E-12


# Matrix Q is
[[ 2.30620202e-22  0.00000000e+00  9.17598743e-23 -9.17598743e-23]
 [ 0.00000000e+00 -4.53317446e-23 -9.17598743e-23  9.17598743e-23]
 [ 0.00000000e+00  0.00000000e+00  7.02557307e-23  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.94442065e-22]]

Minimum energy is  -9.016268676828002e-23



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):2.306202017744289e-22, ('q2','q2'):-4.5331744562550646e-23, ('q3','q3'):7.025573073384645e-23, ('q4','q4'):1.944420652831224e-22}

quadratic = {('q1','q3'):9.175987433661365e-23, ('q1','q4'):-9.175987433

In [245]:
linear = {('q1','q1'):2.306202017744289e-22, ('q2','q2'):-4.5331744562550646e-23, ('q3','q3'):7.025573073384645e-23, ('q4','q4'):1.944420652831224e-22}

quadratic = {('q1','q3'):9.175987433661365e-23, ('q1','q4'):-9.175987433661365e-23, ('q2','q3'):-9.175987433661365e-23, ('q2','q4'):9.175987433661365e-23} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  1  1  0   -0.0     988     0.0
1  0  1  0  0   -0.0      12     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [246]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -38
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727594815078E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106879507657E+01
Previous error of x is 0.00000000000000000000E+00
Previous error of y is -1.49213974509621039033E-12


# Matrix Q is
[[ 1.23757014e-22  0.00000000e+00  9.17598743e-23 -9.17598743e-23]
 [ 0.00000000e+00  6.15314430e-23 -9.17598743e-23  9.17598743e-23]
 [ 0.00000000e+00  0.00000000e+00  2.36766604e-22  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.79311916e-23]]

Minimum energy is  -2.0679515313825692e-23



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.2375701416546083e-22, ('q2','q2'):6.153144304641742e-23, ('q3','q3'):2.3676660444473166e-22, ('q4','q4'):2.793119157223717e-23}

quadratic = {('q1','q3'):9.175987433661365e-23, ('q1','q4'):-9.17598743

In [247]:
linear = {('q1','q1'):1.2375701416546083e-22, ('q2','q2'):6.153144304641742e-23, ('q3','q3'):2.3676660444473166e-22, ('q4','q4'):2.793119157223717e-23}

quadratic = {('q1','q3'):9.175987433661365e-23, ('q1','q4'):-9.175987433661365e-23, ('q2','q3'):-9.175987433661365e-23, ('q2','q4'):9.175987433661365e-23} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     979     0.0
1  0  0  0  1    0.0      21     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [248]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -39
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727594815078E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106879507657E+01
Previous error of x is 0.00000000000000000000E+00
Previous error of y is -1.49213974509621039033E-12


# Matrix Q is
[[ 3.87174499e-23  0.00000000e+00  2.29399686e-23 -2.29399686e-23]
 [ 0.00000000e+00  7.60466437e-24 -2.29399686e-23  2.29399686e-23]
 [ 0.00000000e+00  0.00000000e+00  8.52960777e-23  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.91216287e-23]]

Minimum energy is  -2.0679515313825692e-23



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):3.871744993124564e-23, ('q2','q2'):7.604664371723923e-24, ('q3','q3'):8.529607772024474e-23, ('q4','q4'):-1.9121628716002523e-23}

quadratic = {('q1','q3'):2.2939968584153413e-23, ('q1','q4'):-2.2939968

In [249]:
linear = {('q1','q1'):3.871744993124564e-23, ('q2','q2'):7.604664371723923e-24, ('q3','q3'):8.529607772024474e-23, ('q4','q4'):-1.9121628716002523e-23}

quadratic = {('q1','q3'):2.2939968584153413e-23, ('q1','q4'):-2.2939968584153413e-23, ('q2','q3'):-2.2939968584153413e-23, ('q2','q4'):2.2939968584153413e-23} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  1   -0.0     997     0.0
1  1  0  0  1   -0.0       3     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [250]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -39
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727594815078E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106897697551E+01
Previous error of x is 0.00000000000000000000E+00
Previous error of y is 3.26849658449646085501E-13


# Matrix Q is
[[ 1.57625288e-23  0.00000000e+00  2.29399686e-23 -2.29399686e-23]
 [ 0.00000000e+00  3.05595855e-23 -2.29399686e-23  2.29399686e-23]
 [ 0.00000000e+00  0.00000000e+00  2.43331677e-23  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.18412813e-23]]

Minimum energy is  -8.271806125530277e-25



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):1.5762528834130837e-23, ('q2','q2'):3.0559585468838724e-23, ('q3','q3'):2.433316774152357e-23, ('q4','q4'):4.184128126271864e-23}

quadratic = {('q1','q3'):2.2939968584153413e-23, ('q1','q4'):-2.293996858

In [251]:
linear = {('q1','q1'):1.5762528834130837e-23, ('q2','q2'):3.0559585468838724e-23, ('q3','q3'):2.433316774152357e-23, ('q4','q4'):4.184128126271864e-23}

quadratic = {('q1','q3'):2.2939968584153413e-23, ('q1','q4'):-2.2939968584153413e-23, ('q2','q3'):-2.2939968584153413e-23, ('q2','q4'):2.2939968584153413e-23} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0    1000     0.0
['BINARY', 1 rows, 1000 samples, 4 variables]


In [252]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -40
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727594815078E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106897697551E+01
Previous error of x is 0.00000000000000000000E+00
Previous error of y is 3.26849658449646085501E-13


# Matrix Q is
[[ 2.09100013e-24  0.00000000e+00  5.73499215e-24 -5.73499215e-24]
 [ 0.00000000e+00  9.48952845e-24 -5.73499215e-24  5.73499215e-24]
 [ 0.00000000e+00  0.00000000e+00  3.89477775e-24  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.26488345e-23]]

Minimum energy is  -8.271806125530277e-25



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):2.0910001291942235e-24, ('q2','q2'):9.489528446548167e-24, ('q3','q3'):3.894777745231508e-24, ('q4','q4'):1.2648834505829045e-23}

quadratic = {('q1','q3'):5.734992146038353e-24, ('q1','q4'):-5.7349921460

In [253]:
linear = {('q1','q1'):2.0910001291942235e-24, ('q2','q2'):9.489528446548167e-24, ('q3','q3'):3.894777745231508e-24, ('q4','q4'):1.2648834505829045e-23}

quadratic = {('q1','q3'):5.734992146038353e-24, ('q1','q4'):-5.734992146038353e-24, ('q2','q3'):-5.734992146038353e-24, ('q2','q4'):5.734992146038353e-24} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     997     0.0
1  1  0  0  0    0.0       3     0.0
['BINARY', 2 rows, 1000 samples, 4 variables]


In [254]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -41
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727594815078E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106897697551E+01
Previous error of x is 0.00000000000000000000E+00
Previous error of y is 3.26849658449646085501E-13


# Matrix Q is
[[-4.02066007e-25  0.00000000e+00  1.43374804e-24 -1.43374804e-24]
 [ 0.00000000e+00  3.29719815e-24 -1.43374804e-24  1.43374804e-24]
 [ 0.00000000e+00  0.00000000e+00 -1.20562659e-25  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.25646572e-24]]

Minimum energy is  -8.271806125530277e-25



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-4.0206600737068704e-25, ('q2','q2'):3.2971981513062846e-24, ('q3','q3'):-1.2056265876681503e-25, ('q4','q4'):4.2564657215319534e-24}

quadratic = {('q1','q3'):1.4337480365095883e-24, ('q1','q4'):-1.43374

In [255]:
linear = {('q1','q1'):-4.0206600737068704e-25, ('q2','q2'):3.2971981513062846e-24, ('q3','q3'):-1.2056265876681503e-25, ('q4','q4'):4.2564657215319534e-24}

quadratic = {('q1','q3'):1.4337480365095883e-24, ('q1','q4'):-1.4337480365095883e-24, ('q2','q3'):-1.4337480365095883e-24, ('q2','q4'):1.4337480365095883e-24} 
Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  1  0  0  0   -0.0     854     0.0
1  0  0  1  0   -0.0     114     0.0
2  0  0  0  0    0.0      31     0.0
3  1  0  1  0    0.0       1     0.0
['BINARY', 4 rows, 1000 samples, 4 variables]


In [256]:
Samples = sampleset.samples()
qb_data = Samples[0,['q1','q2','q3','q4']]
q1p = pow(2,Curr_Pow)*(qb_data[0])
q1n = -pow(2,Curr_Pow)*(qb_data[1])
q2p = pow(2,Curr_Pow)*(qb_data[2])
q2n = -pow(2,Curr_Pow)*(qb_data[3])

val_q1 = q1p + q1n
val_q2 = q2p + q2n

if val_q1 == 0 and val_q2 == 0:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    Center = pre_Center
    Curr_Pow = Pre_Pow -qubits
else:
    Pre_Pow = Curr_Pow
    pre_Center = Center
    add_Center = np.array([val_q1, val_q2])
    Center = pre_Center + add_Center
    Curr_Pow = Pre_Pow 
    
c = b - np.dot(A,Center)
(Dimension, Dimension_same) = A.shape

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*(l+Curr_Pow))*pow(A[k][i],2)
            cef2 = pow(2,(l+Curr_Pow)+1)*A[k][i]*c[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, (l1+Curr_Pow)+(l2+Curr_Pow)+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef


# Print Matrix Q
print("Current Power is ", Curr_Pow)
print("Solution x is %20.20E" % x_fin[0])
print("Current  x is %20.20E" % Center[0])
print("Solution y is %20.20E" % x_fin[1])
print("Current  y is %20.20E" % Center[1])
print("Previous error of x is %20.20E" % (x_fin[0] - Center[0]))
print("Previous error of y is %20.20E" % (x_fin[1] - Center[1]))
print("\n\n# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(2*qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            elif i == 2*qubits*Dimension-3 and j == 2*qubits*Dimension-1:
                if QM[i+1][j] != 0:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")
                else:
                    print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = "} ")  
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")   

Current Power is  -41
Solution x is 3.21699087727594815078E+03
Current  x is 3.21699087727594860553E+03
Solution y is -8.69850185106894429055E+01
Current  y is -8.69850185106897697551E+01
Previous error of x is -4.54747350886464118958E-13
Previous error of y is 3.26849658449646085501E-13


# Matrix Q is
[[ 2.61737611e-24  0.00000000e+00  1.43374804e-24 -1.43374804e-24]
 [ 0.00000000e+00  2.77756031e-25 -1.43374804e-24  1.43374804e-24]
 [ 0.00000000e+00  0.00000000e+00  6.35232683e-25  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.50067038e-24]]

Minimum energy is  -8.271806125530277e-25



Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):2.6173761127723752e-24, ('q2','q2'):2.7775603116322245e-25, ('q3','q3'):6.352326834047792e-25, ('q4','q4'):3.50067037936036e-24}

quadratic = {('q1','q3'):1.4337480365095883e-24, ('q1','q4'):-1.433748036